<h1 style="color: #1e3a8a; background-color: #dbeafe; padding: 12px; border-left: 5px solid #2563eb; margin: 20px 0; font-weight: bold;">Modélisation Home Credit Default Risk</h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, cross_validate
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, 
    f1_score, confusion_matrix, classification_report, roc_curve,
    precision_recall_curve
)
import shap

# Modèles
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# Optimisation
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)

# Outils système
import logging
import time
import warnings
warnings.filterwarnings('ignore')
import subprocess

# Configuration affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')

# Suivi des expériences et gestion des modèles avec MLflow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Activer autolog pour sklearn et lightgbm
mlflow.sklearn.autolog(disable=True)  # Désactivé car logging manuel plus complet
mlflow.lightgbm.autolog(disable=True)  # Désactivé car logging manuel plus complet

<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">MLflow - Infrastructure</h2>

In [2]:
# CONFIGURATION MLFLOW - SQLITE

print("Configuration MLflow avec Backend SQLite")
print("="*70)

# Tracking URI - Base SQLite locale
tracking_uri = "sqlite:///mlflow.db"
mlflow.set_tracking_uri(tracking_uri)

print(f"Tracking URI configuré : {mlflow.get_tracking_uri()}")
print(f"Base de données : mlflow.db (racine du projet)")
print(f"Artifacts : ./mlruns/ (créé automatiquement)")

Configuration MLflow avec Backend SQLite
Tracking URI configuré : sqlite:///mlflow.db
Base de données : mlflow.db (racine du projet)
Artifacts : ./mlruns/ (créé automatiquement)


In [11]:
# CRÉATION EXPÉRIENCE PRINCIPALE

# Nom de l'expérience
experiment_name = "Home_Credit_Default"

print(f"\nConfiguration expérience : {experiment_name}")
print("="*70)

# Vérifier si l'expérience existe déjà
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    # Créer l'expérience avec métadonnées
    experiment_id = mlflow.create_experiment(
        experiment_name,
        tags={
            "project": "Prêt à dépenser - Credit Scoring",
            "team": "Data Science",
            "author": "Mounir Meknaci",
            "objective": "Prédire probabilité défaut de paiement",
            "dataset_version": "v1.0_preprocessed",
            "features_count": "646 colonnes (avant encodage)",
            "target_distribution": "91.9% classe 0 / 8.1% classe 1",
            "class_imbalance_ratio": "11.4:1",
            "business_constraint": "Coût FN = 10x Coût FP",
            "target_metric": "ROC-AUC < 0.82 (éviter overfitting)"
        }
    )
    print(f"Expérience créée : {experiment_name}")
    print(f"Experiment ID : {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Expérience existante trouvée : {experiment_name}")
    print(f"Experiment ID : {experiment_id}")

# Définir comme expérience active
mlflow.set_experiment(experiment_name)
print(f"Expérience active : {experiment_name}")


Configuration expérience : Home_Credit_Default
Expérience existante trouvée : Home_Credit_Default
Experiment ID : 1
Expérience active : Home_Credit_Default


In [12]:
# VÉRIFICATION INFRASTRUCTURE

print(f"\nVérification Infrastructure MLflow")
print("="*70)

# Client MLflow
client = MlflowClient()

# Lister toutes les expériences
experiments = client.search_experiments()
print(f"\nExpériences disponibles : {len(experiments)}")
for exp in experiments:
    if exp.lifecycle_stage == "active":
        print(f"  • {exp.name} (ID: {exp.experiment_id})")

# Détails expérience active
active_exp = mlflow.get_experiment_by_name(experiment_name)
print(f"\nEXPÉRIENCE ACTIVE :")
print(f"  • Nom : {active_exp.name}")
print(f"  • ID : {active_exp.experiment_id}")
print(f"  • Artifact Location : {active_exp.artifact_location}")
print(f"  • Lifecycle : {active_exp.lifecycle_stage}")

print(f"\nInfrastructure MLflow opérationnelle")
print(f"Pour lancer l'UI MLflow :")
print(f"   Terminal → mlflow ui --backend-store-uri sqlite:///mlflow.db")
print(f"   Browser → http://127.0.0.1:5000")


Vérification Infrastructure MLflow

Expériences disponibles : 3
  • home_credit_scoring (ID: 2)
  • Home_Credit_Default (ID: 1)
  • Default (ID: 0)

EXPÉRIENCE ACTIVE :
  • Nom : Home_Credit_Default
  • ID : 1
  • Artifact Location : /Users/mounirmeknaci/Desktop/Data_Projects/Projet6/mlruns/1
  • Lifecycle : active

Infrastructure MLflow opérationnelle
Pour lancer l'UI MLflow :
   Terminal → mlflow ui --backend-store-uri sqlite:///mlflow.db
   Browser → http://127.0.0.1:5000


In [13]:
# CHARGEMENT DATASET PRÉPARÉ

print("\nChargement des données")
print("="*70)

# Charger le dataset
df_app_modelisation = pd.read_csv('app_train_models.csv')

print(f"Dataset chargé : app_train_models.csv")

# Aperçu des données
print(f"\n=== APERÇU DES DONNÉES ===")

print(f"Forme : {df_app_modelisation.shape}")
print(f"Colonnes : {df_app_modelisation.shape[1]}")
print(f"Observations : {df_app_modelisation.shape[0]}")
print("\nPremières lignes :")
df_app_modelisation.head()

print(f"Premières colonnes: {list(df_app_modelisation.columns[:5])}")
print(f"Dernières colonnes: {list(df_app_modelisation.columns[-5:])}")

# Vérification de la target
if 'TARGET' in df_app_modelisation.columns:
    target_dist = df_app_modelisation['TARGET'].value_counts(normalize=True)
    print(f"\nDISTRIBUTION DE LA TARGET:")
    print(f"Classe 0 (pas de défaut): {target_dist[0]:.1%}")
    print(f"Classe 1 (défaut): {target_dist[1]:.1%}")

# Vérification des valeurs manquantes
missing_count = df_app_modelisation.isnull().sum().sum()
print(f"\nQUALITÉ DES DONNÉES:")
print(f"Valeurs manquantes: {missing_count}")
print(f"Types de données: {df_app_modelisation.dtypes.value_counts().to_dict()}")

# Variables catégorielles à encoder
categorical_cols = df_app_modelisation.select_dtypes(include=['object']).columns.tolist()
print(f"\nVARIABLES CATÉGORIELLES À ENCODER: {len(categorical_cols)}")
if len(categorical_cols) > 0:
    for col in categorical_cols:
        n_categories = df_app_modelisation[col].nunique()
        strategy = "Label Encoding" if n_categories <= 2 else "One-Hot Encoding"
        print(f"  • {col}: {n_categories} catégories → {strategy}")


Chargement des données
Dataset chargé : app_train_models.csv

=== APERÇU DES DONNÉES ===
Forme : (307511, 646)
Colonnes : 646
Observations : 307511

Premières lignes :
Premières colonnes: ['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR']
Dernières colonnes: ['EXT_SOURCE_2 EXT_SOURCE_3', 'EXT_SOURCE_2 DAYS_BIRTH', 'EXT_SOURCE_3^2', 'EXT_SOURCE_3 DAYS_BIRTH', 'DAYS_BIRTH^2']

DISTRIBUTION DE LA TARGET:
Classe 0 (pas de défaut): 91.9%
Classe 1 (défaut): 8.1%

QUALITÉ DES DONNÉES:
Valeurs manquantes: 0
Types de données: {dtype('float64'): 566, dtype('int64'): 43, dtype('O'): 37}

VARIABLES CATÉGORIELLES À ENCODER: 37
  • NAME_CONTRACT_TYPE: 2 catégories → Label Encoding
  • CODE_GENDER: 3 catégories → One-Hot Encoding
  • FLAG_OWN_CAR: 2 catégories → Label Encoding
  • FLAG_OWN_REALTY: 2 catégories → Label Encoding
  • NAME_TYPE_SUITE: 7 catégories → One-Hot Encoding
  • NAME_INCOME_TYPE: 8 catégories → One-Hot Encoding
  • NAME_EDUCATION_TYPE: 5 catégories → O

<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Split Train/Validation avec Stratification</h2>

In [17]:
# Split train/validation avec stratification
print("=== SPLIT TRAIN/VALIDATION STRATIFIÉ ===")

# Séparer features et target
X = df_app_modelisation.drop(['TARGET'], axis=1)
y = df_app_modelisation['TARGET']

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")

# Split stratifié 80/20
test_size = 0.2
random_state = 42

X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=test_size, 
    random_state=random_state,
    stratify=y  # STRATIFICATION pour préserver les proportions
)

print(f"\n=== RÉSULTATS DU SPLIT ===")
print(f"Train set: {X_train.shape[0]:,} échantillons ({(1-test_size)*100:.0f}%)")
print(f"Validation set: {X_val.shape[0]:,} échantillons ({test_size*100:.0f}%)")

# Vérifier la préservation des proportions
original_proportions = y.value_counts(normalize=True)
train_proportions = y_train.value_counts(normalize=True)
val_proportions = y_val.value_counts(normalize=True)

print(f"\nVÉRIFICATION STRATIFICATION:")
print(f"{'Set':<12} {'Classe 0':<10} {'Classe 1':<10} {'Ratio':<10}")
print("-" * 50)
print(f"{'Original':<12} {original_proportions[0]:<10.1%} {original_proportions[1]:<10.1%} {original_proportions[0]/original_proportions[1]:<10.1f}")
print(f"{'Train':<12} {train_proportions[0]:<10.1%} {train_proportions[1]:<10.1%} {train_proportions[0]/train_proportions[1]:<10.1f}")
print(f"{'Validation':<12} {val_proportions[0]:<10.1%} {val_proportions[1]:<10.1%} {val_proportions[0]/val_proportions[1]:<10.1f}")

=== SPLIT TRAIN/VALIDATION STRATIFIÉ ===
Features (X): (307511, 645)
Target (y): (307511,)

=== RÉSULTATS DU SPLIT ===
Train set: 246,008 échantillons (80%)
Validation set: 61,503 échantillons (20%)

VÉRIFICATION STRATIFICATION:
Set          Classe 0   Classe 1   Ratio     
--------------------------------------------------
Original     91.9%      8.1%       11.4      
Train        91.9%      8.1%       11.4      
Validation   91.9%      8.1%       11.4      


<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Encodage des Variables Catégorielles</h2>

**Stratégie d'encodage :**
- **≤2 catégories** → Label Encoding (0/1)
- **>2 catégories** → One-Hot Encoding (variables binaires)
- **Fit sur train uniquement** → Transform sur train et validation

In [18]:
# Encodage des variables catégorielles avec stratégie anti-leakage
print("=== ENCODAGE DES VARIABLES CATÉGORIELLES ===")

# Identifier les variables catégorielles
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
print(f"Variables catégorielles détectées: {len(categorical_cols)}")

if len(categorical_cols) == 0:
    print("Aucune variable catégorielle à encoder - Datasets déjà numériques")
    X_train_encoded = X_train.copy()
    X_val_encoded = X_val.copy()
else:
    # Analyser chaque variable catégorielle
    print(f"\n=== ANALYSE DES VARIABLES CATÉGORIELLES ===")
    cols_label_encoding = []
    cols_onehot_encoding = []
    
    for col in categorical_cols:
        n_categories = X_train[col].nunique()
        if n_categories <= 2:
            cols_label_encoding.append(col)
            strategy = "Label Encoding"
        else:
            cols_onehot_encoding.append(col)
            strategy = "One-Hot Encoding"
        print(f"  • {col}: {n_categories} catégories → {strategy}")
    
    print(f"\nSTRATÉGIE D'ENCODAGE:")
    print(f"Label Encoding (≤2 cat): {len(cols_label_encoding)}")
    print(f"One-Hot Encoding (>2 cat): {len(cols_onehot_encoding)}")
    
    # Copier les datasets pour encodage
    X_train_encoded = X_train.copy()
    X_val_encoded = X_val.copy()
    
    # Dictionnaires pour stocker les encodeurs
    label_encoders = {}
    onehot_encoders = {}
    
    # LABEL ENCODING pour variables ≤2 catégories
    if len(cols_label_encoding) > 0:
        print(f"\nAPPLICATION LABEL ENCODING:")
        
        for col in cols_label_encoding:
            print(f"  Encodage: {col}")
            
            # Fit sur TRAIN uniquement
            le = LabelEncoder()
            le.fit(X_train_encoded[col])
            
            # Transform train
            X_train_encoded[col] = le.transform(X_train_encoded[col])
            
            # Transform validation (gestion catégories nouvelles)
            val_unknown = ~X_val_encoded[col].isin(le.classes_)
            if val_unknown.sum() > 0:
                print(f"{val_unknown.sum()} catégories nouvelles en validation")
                # Remplacer par la plus fréquente du train
                most_frequent = X_train[col].mode()[0]
                X_val_encoded.loc[val_unknown, col] = most_frequent
            
            X_val_encoded[col] = le.transform(X_val_encoded[col])
            label_encoders[col] = le
    
    # ONE-HOT ENCODING pour variables >2 catégories
    if len(cols_onehot_encoding) > 0:
        print(f"\nAPPLICATION ONE-HOT ENCODING:")
        
        for col in cols_onehot_encoding:
            print(f"  Encodage: {col}")
            
            # Fit sur TRAIN uniquement
            ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            ohe.fit(X_train_encoded[[col]])
            
            # Créer les noms des nouvelles colonnes
            feature_names = [f"{col}_{cat}" for cat in ohe.categories_[0]]
            
            # Transform TRAIN
            train_encoded = ohe.transform(X_train_encoded[[col]])
            train_ohe_df = pd.DataFrame(train_encoded, columns=feature_names, index=X_train_encoded.index)
            X_train_encoded = X_train_encoded.drop(columns=[col])
            X_train_encoded = pd.concat([X_train_encoded, train_ohe_df], axis=1)
            
            # Transform VALIDATION
            val_encoded = ohe.transform(X_val_encoded[[col]])
            val_ohe_df = pd.DataFrame(val_encoded, columns=feature_names, index=X_val_encoded.index)
            X_val_encoded = X_val_encoded.drop(columns=[col])
            X_val_encoded = pd.concat([X_val_encoded, val_ohe_df], axis=1)
            
            onehot_encoders[col] = ohe
    
    print(f"\nENCODAGE TERMINÉ:")
    print(f"  • Label Encoders créés: {len(label_encoders)}")
    print(f"  • One-Hot Encoders créés: {len(onehot_encoders)}")
    
    # Calculer nouvelles features ajoutées par One-Hot
    if len(cols_onehot_encoding) > 0:
        original_cols = len(X_train.columns)
        new_cols = len(X_train_encoded.columns)
        onehot_features_added = new_cols - original_cols + len(cols_onehot_encoding)
        print(f"  • Features One-Hot ajoutées: {onehot_features_added}")

# Vérification finale
remaining_objects_train = X_train_encoded.select_dtypes(include=['object']).columns.tolist()
remaining_objects_val = X_val_encoded.select_dtypes(include=['object']).columns.tolist()

print(f"\n=== VÉRIFICATION POST-ENCODAGE ===")
print(f"Variables 'object' restantes:")
print(f"  Train: {len(remaining_objects_train)}")
print(f"  Validation: {len(remaining_objects_val)}")

if len(remaining_objects_train) == 0 and len(remaining_objects_val) == 0:
    print("ENCODAGE COMPLET - Tous datasets 100% numériques")
else:
    print("Variables object restantes à traiter")
    if remaining_objects_train:
        print(f"  Train: {remaining_objects_train}")
    if remaining_objects_val:
        print(f"  Validation: {remaining_objects_val}")

print(f"\nDatasets finaux après encodage:")
print(f"Train: {X_train_encoded.shape}")
print(f"Validation: {X_val_encoded.shape}")

print(f"\nENCODAGE TERMINÉ - DATASETS PRÊTS POUR MODÉLISATION")
print(f"Stratégie appliquée: Label Encoding (≤2 cat) + One-Hot Encoding (>2 cat)")

=== ENCODAGE DES VARIABLES CATÉGORIELLES ===
Variables catégorielles détectées: 37

=== ANALYSE DES VARIABLES CATÉGORIELLES ===
  • NAME_CONTRACT_TYPE: 2 catégories → Label Encoding
  • CODE_GENDER: 3 catégories → One-Hot Encoding
  • FLAG_OWN_CAR: 2 catégories → Label Encoding
  • FLAG_OWN_REALTY: 2 catégories → Label Encoding
  • NAME_TYPE_SUITE: 7 catégories → One-Hot Encoding
  • NAME_INCOME_TYPE: 8 catégories → One-Hot Encoding
  • NAME_EDUCATION_TYPE: 5 catégories → One-Hot Encoding
  • NAME_FAMILY_STATUS: 6 catégories → One-Hot Encoding
  • NAME_HOUSING_TYPE: 6 catégories → One-Hot Encoding
  • OCCUPATION_TYPE: 18 catégories → One-Hot Encoding
  • WEEKDAY_APPR_PROCESS_START: 7 catégories → One-Hot Encoding
  • ORGANIZATION_TYPE: 58 catégories → One-Hot Encoding
  • FONDKAPREMONT_MODE: 4 catégories → One-Hot Encoding
  • HOUSETYPE_MODE: 3 catégories → One-Hot Encoding
  • WALLSMATERIAL_MODE: 7 catégories → One-Hot Encoding
  • EMERGENCYSTATE_MODE: 2 catégories → Label Encoding
  

<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Test infrastructure MLflow </h2>

<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">DummyClassifier</h3>

In [19]:
# DUMMYCLASSIFIER (Test - dans Home_Credit_Default)

df_train = df_app_modelisation

# Démarrer run MLflow
with mlflow.start_run(run_name="Baseline_Dummy_MostFrequent") as run:
    
    print(f"Run MLflow démarré")
    print(f"Run ID : {run.info.run_id}")
    print(f"Experiment ID : {run.info.experiment_id}")
    print()
    
    start_time = time.time()
    
    # ---------- MODÈLE ----------
    print("Entraînement DummyClassifier (strategy='most_frequent')...")
    model = DummyClassifier(strategy='most_frequent', random_state=42)
    
    # Log hyperparamètres
    mlflow.log_param("model_type", "DummyClassifier")
    mlflow.log_param("strategy", "most_frequent")
    mlflow.log_param("random_state", 42)
    mlflow.log_param("n_features", X_train_encoded.shape[1])
    mlflow.log_param("n_train_samples", X_train_encoded.shape[0])
    mlflow.log_param("n_val_samples", X_val_encoded.shape[0])
    mlflow.log_param("class_weight", "None")
    
    # Entraînement
    model.fit(X_train_encoded, y_train)
    training_time = time.time() - start_time
    
    # ---------- PRÉDICTIONS ----------
    y_pred = model.predict(X_val_encoded)
    y_proba = model.predict_proba(X_val_encoded)[:, 1]
    
    # ---------- MÉTRIQUES ----------
    print("Calcul des métriques...")
    
    # Métriques principales
    auc = roc_auc_score(y_val, y_proba)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, zero_division=0)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, zero_division=0)
    
    # Métriques classe 0
    precision_0 = precision_score(y_val, y_pred, pos_label=0, zero_division=0)
    recall_0 = recall_score(y_val, y_pred, pos_label=0)
    f1_0 = f1_score(y_val, y_pred, pos_label=0, zero_division=0)
    
    # Confusion matrix
    cm = confusion_matrix(y_val, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Log toutes les métriques
    mlflow.log_metric("auc_roc", auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision_class_1", precision)
    mlflow.log_metric("recall_class_1", recall)
    mlflow.log_metric("f1_score_class_1", f1)
    mlflow.log_metric("precision_class_0", precision_0)
    mlflow.log_metric("recall_class_0", recall_0)
    mlflow.log_metric("f1_score_class_0", f1_0)
    mlflow.log_metric("true_negatives", int(tn))
    mlflow.log_metric("false_positives", int(fp))
    mlflow.log_metric("false_negatives", int(fn))
    mlflow.log_metric("true_positives", int(tp))
    mlflow.log_metric("training_time_seconds", training_time)
    
    # ---------- ARTEFACTS ----------
    print("Génération des artéfacts...")
    
    # Confusion Matrix
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Pas Défaut (0)', 'Défaut (1)'],
                yticklabels=['Pas Défaut (0)', 'Défaut (1)'],
                cbar_kws={'label': 'Nombre de prédictions'})
    ax.set_title(f'Matrice de Confusion - DummyClassifier\nAUC: {auc:.4f}', 
                fontsize=14, fontweight='bold')
    ax.set_ylabel('Vraie Classe', fontsize=12)
    ax.set_xlabel('Classe Prédite', fontsize=12)
    plt.tight_layout()
    plt.savefig("graphiques/confusion_matrix.png", dpi=100, bbox_inches='tight')
    mlflow.log_artifact("graphiques/confusion_matrix.png")
    plt.close()
    
    # ROC Curve
    fpr, tpr, thresholds_roc = roc_curve(y_val, y_proba)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(fpr, tpr, label=f'DummyClassifier (AUC = {auc:.4f})', 
           linewidth=2, color='#3498db')
    ax.plot([0, 1], [0, 1], 'k--', label='Baseline (AUC = 0.50)', linewidth=1)
    ax.set_xlabel('Taux de Faux Positifs (FPR)', fontsize=12)
    ax.set_ylabel('Taux de Vrais Positifs (TPR)', fontsize=12)
    ax.set_title('Courbe ROC - DummyClassifier', fontsize=14, fontweight='bold')
    ax.legend(loc='lower right', fontsize=10)
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig("graphiques/roc_curve.png", dpi=100, bbox_inches='tight')
    mlflow.log_artifact("graphiques/roc_curve.png")
    plt.close()
    
    # Precision-Recall Curve
    precision_curve, recall_curve, thresholds_pr = precision_recall_curve(y_val, y_proba)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(recall_curve, precision_curve, linewidth=2, color='#e74c3c')
    ax.set_xlabel('Recall', fontsize=12)
    ax.set_ylabel('Precision', fontsize=12)
    ax.set_title('Courbe Precision-Recall - DummyClassifier', 
                fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig("graphiques/precision_recall_curve.png", dpi=100, bbox_inches='tight')
    mlflow.log_artifact("graphiques/precision_recall_curve.png")
    plt.close()
    
    # Classification Report
    report = classification_report(
        y_val, y_pred, 
        target_names=['Pas Défaut (0)', 'Défaut (1)'],
        digits=4
    )
    with open("classification_report.txt", 'w') as f:
        f.write("="*60 + "\n")
        f.write("CLASSIFICATION REPORT - DummyClassifier\n")
        f.write("="*60 + "\n\n")
        f.write(report)
        f.write("\n" + "="*60 + "\n")
        f.write(f"Training Time: {training_time:.2f} seconds\n")
    mlflow.log_artifact("classification_report.txt")
    
    # ---------- SAUVEGARDER MODÈLE ----------
    print("Sauvegarde du modèle...")
    
    # Créer un exemple d'input pour la signature
    input_example = X_train_encoded.head(5)
    
    # Obtenir la version scikit-learn
    import sklearn
    sklearn_version = sklearn.__version__
    
    # Log modèle avec nouvelle syntaxe MLflow 3.4.0
    mlflow.sklearn.log_model(
        model,                          # Premier argument (pas sk_model)
        "model",                        # Artifact path (pas artifact_path=)
        input_example=input_example,    # Pour signature
        pip_requirements=[              # Supprime warning pip
            f"scikit-learn=={sklearn_version}",
            "numpy",
            "pandas"
        ]
    )
    
    # ---------- TAGS ----------
    mlflow.set_tags({
        "model_family": "baseline",
        "class_handling": "none",
        "validation_strategy": "single_split_80_20",
        "feature_engineering": "complete",
        "purpose": "infrastructure_test"
    })
    
    # ---------- RÉSUMÉ ----------
    print()
    print("="*70)
    print("RÉSULTATS DU RUN")
    print("="*70)
    print(f"Modèle         : DummyClassifier (most_frequent)")
    print(f"Training Time  : {training_time:.2f}s")
    print(f"AUC-ROC        : {auc:.4f}")
    print(f"Accuracy       : {accuracy:.4f}")
    print(f"Recall (Cl. 1) : {recall:.4f}")
    print(f"Recall (Cl. 0) : {recall_0:.4f}")
    print(f"F1-Score       : {f1:.4f}")
    print(f"Confusion Matrix :")
    print(f"     TN: {tn:,}  |  FP: {fp:,}")
    print(f"     FN: {fn:,}  |  TP: {tp:,}")
    print(f"Run ID         : {run.info.run_id}")
    print("="*70)
    
    print(f"\nRun terminé et sauvegardé dans MLflow")
    print(f"Artéfacts sauvegardés : 4 fichiers")
    print(f"   • confusion_matrix.png")
    print(f"   • roc_curve.png")
    print(f"   • precision_recall_curve.png")
    print(f"   • classification_report.txt")

print("\n" + "="*70)
print("TEST INFRASTRUCTURE MLFLOW RÉUSSI")
print("="*70)
print()
print("LANCER L'INTERFACE MLFLOW UI :")
print("   1. Ouvrir un terminal dans le dossier Projet6/")
print("   2. Exécuter : mlflow ui --backend-store-uri sqlite:///mlflow.db")
print("   3. Ouvrir navigateur : http://localhost:5000")
print()
print("PROCHAINES ÉTAPES :")
print("   • Vérifier le run dans l'UI MLflow")
print("   • Confirmer présence des artéfacts")
print("   • Passer aux modèles baseline (LogReg, RF, GBM)")

Run MLflow démarré
Run ID : 4843a4759c38472e8930770c96f3aa9e
Experiment ID : 2

Entraînement DummyClassifier (strategy='most_frequent')...
Calcul des métriques...
Génération des artéfacts...


2025/10/20 13:04:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Sauvegarde du modèle...


2025/10/20 13:04:10 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



RÉSULTATS DU RUN
Modèle         : DummyClassifier (most_frequent)
Training Time  : 0.02s
AUC-ROC        : 0.5000
Accuracy       : 0.9193
Recall (Cl. 1) : 0.0000
Recall (Cl. 0) : 1.0000
F1-Score       : 0.0000
Confusion Matrix :
     TN: 56,538  |  FP: 0
     FN: 4,965  |  TP: 0
Run ID         : 4843a4759c38472e8930770c96f3aa9e

Run terminé et sauvegardé dans MLflow
Artéfacts sauvegardés : 4 fichiers
   • confusion_matrix.png
   • roc_curve.png
   • precision_recall_curve.png
   • classification_report.txt

TEST INFRASTRUCTURE MLFLOW RÉUSSI

LANCER L'INTERFACE MLFLOW UI :
   1. Ouvrir un terminal dans le dossier Projet6/
   2. Exécuter : mlflow ui --backend-store-uri sqlite:///mlflow.db
   3. Ouvrir navigateur : http://localhost:5000

PROCHAINES ÉTAPES :
   • Vérifier le run dans l'UI MLflow
   • Confirmer présence des artéfacts
   • Passer aux modèles baseline (LogReg, RF, GBM)


In [20]:
# Vérification de sauvegarde du modèle

# Récupérer le dernier run
client = MlflowClient()
experiment = client.get_experiment_by_name("Home_Credit_Default")
runs = client.search_runs(experiment.experiment_id, order_by=["start_time DESC"], max_results=1)

if runs:
    last_run = runs[0]
    run_id = last_run.info.run_id
    
    print(f"Dernier Run ID : {run_id}")
    
    # Charger le modèle sauvegardé
    model_uri = f"runs:/{run_id}/model"
    loaded_model = mlflow.sklearn.load_model(model_uri)
    
    print(f"Modèle chargé avec succès !")
    print(f"Type : {type(loaded_model)}")
    
    # Test prédiction
    test_pred = loaded_model.predict(X_val_encoded.head(10))
    print(f"Prédictions test : {test_pred}")
    
    print("\nVOTRE MODÈLE FONCTIONNE PARFAITEMENT")
else:
    print("Aucun run trouvé")

Dernier Run ID : 3cbd34060fd4431c888d47bfe36c91e9


Modèle chargé avec succès !
Type : <class 'sklearn.dummy.DummyClassifier'>
Prédictions test : [0 0 0 0 0 0 0 0 0 0]

VOTRE MODÈLE FONCTIONNE PARFAITEMENT


<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Modélisation</h2>

<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Fonction réutilisable </h3>

In [21]:
# Fonction réutilisable avec optimisation du coût métier et du seuil

def calculate_business_cost(y_true, y_pred, cost_fn=10, cost_fp=1):
    """
    Calcule le coût métier basé sur la matrice de confusion.
    
    Parameters:
    -----------
    y_true : array-like
        Vraies classes
    y_pred : array-like
        Classes prédites
    cost_fn : int, default=10
        Coût d'un Faux Négatif (défaut non détecté)
    cost_fp : int, default=1
        Coût d'un Faux Positif (bon client refusé)
    
    Returns:
    --------
    dict : Dictionnaire avec les coûts détaillés
    """
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    cost_false_negatives = fn * cost_fn
    cost_false_positives = fp * cost_fp
    total_cost = cost_false_negatives + cost_false_positives
    cost_per_prediction = total_cost / len(y_true)
    
    return {
        "cost_fn_total": cost_false_negatives,
        "cost_fp_total": cost_false_positives,
        "total_business_cost": total_cost,
        "cost_per_prediction": cost_per_prediction,
        "fn_count": int(fn),
        "fp_count": int(fp),
        "tn_count": int(tn),
        "tp_count": int(tp)
    }


def optimize_threshold(y_true, y_proba, cost_fn=10, cost_fp=1, threshold_range=(0.05, 0.95), threshold_step=0.01):
    """
    Trouve le seuil optimal qui minimise le coût métier.
    
    Parameters:
    -----------
    y_true : array-like
        Vraies classes
    y_proba : array-like
        Probabilités prédites pour la classe positive
    cost_fn : int, default=10
        Coût d'un Faux Négatif
    cost_fp : int, default=1
        Coût d'un Faux Positif
    threshold_range : tuple, default=(0.05, 0.95)
        Range des seuils à tester (min, max)
    threshold_step : float, default=0.01
        Pas entre chaque seuil testé
    
    Returns:
    --------
    dict : Dictionnaire avec le seuil optimal et toutes les métriques
    """
    thresholds = np.arange(threshold_range[0], threshold_range[1] + threshold_step, threshold_step)
    
    results = {
        'thresholds': [],
        'costs': [],
        'recalls': [],
        'precisions': [],
        'f1_scores': [],
        'fn_counts': [],
        'fp_counts': []
    }
    
    for threshold in thresholds:
        y_pred_threshold = (y_proba >= threshold).astype(int)
        
        cost_result = calculate_business_cost(y_true, y_pred_threshold, cost_fn, cost_fp)
        
        recall = recall_score(y_true, y_pred_threshold, zero_division=0)
        precision = precision_score(y_true, y_pred_threshold, zero_division=0)
        f1 = f1_score(y_true, y_pred_threshold, zero_division=0)
        
        results['thresholds'].append(threshold)
        results['costs'].append(cost_result['total_business_cost'])
        results['recalls'].append(recall)
        results['precisions'].append(precision)
        results['f1_scores'].append(f1)
        results['fn_counts'].append(cost_result['fn_count'])
        results['fp_counts'].append(cost_result['fp_count'])
    
    optimal_idx = np.argmin(results['costs'])
    optimal_threshold = results['thresholds'][optimal_idx]
    
    y_pred_optimal = (y_proba >= optimal_threshold).astype(int)
    cm_optimal = confusion_matrix(y_true, y_pred_optimal)
    tn_opt, fp_opt, fn_opt, tp_opt = cm_optimal.ravel()
    
    return {
        'optimal_threshold': optimal_threshold,
        'cost_at_optimal': results['costs'][optimal_idx],
        'recall_at_optimal': results['recalls'][optimal_idx],
        'precision_at_optimal': results['precisions'][optimal_idx],
        'f1_at_optimal': results['f1_scores'][optimal_idx],
        'fn_at_optimal': results['fn_counts'][optimal_idx],
        'fp_at_optimal': results['fp_counts'][optimal_idx],
        'tn_at_optimal': int(tn_opt),
        'tp_at_optimal': int(tp_opt),
        'cost_fn_optimal': results['fn_counts'][optimal_idx] * cost_fn,
        'cost_fp_optimal': results['fp_counts'][optimal_idx] * cost_fp,
        'all_thresholds': results['thresholds'],
        'all_costs': results['costs'],
        'all_recalls': results['recalls'],
        'all_precisions': results['precisions'],
        'all_f1_scores': results['f1_scores']
    }


def train_and_log_model(
    model,
    model_name,
    X_train,
    y_train,
    X_val,
    y_val,
    cv_folds=5,
    tags=None,
    description=None,
    optimize_threshold_flag=True,
    cost_fn=10,
    cost_fp=1,
    threshold_range=(0.05, 0.95),
    threshold_step=0.01
):
    """
    Entraine un modele avec Cross-Validation stratifiee et log tout dans MLflow.
    Inclut le calcul du cout metier et l'optimisation du seuil de decision.
    
    FONCTIONNALITES :
    - Cross-validation stratifiee complete
    - Evaluation sur validation set
    - Calcul du cout metier (FN = 10x FP par defaut)
    - Optimisation automatique du seuil de decision
    - Logging complet dans MLflow
    - Generation de visualisations
    - Affichage detaille de toutes les metriques
    
    Parameters:
    -----------
    model : estimator sklearn
        Modele a entrainer (doit avoir predict_proba)
    model_name : str
        Nom du run MLflow
    X_train, y_train : arrays
        Donnees d'entrainement
    X_val, y_val : arrays
        Donnees de validation
    cv_folds : int, default=5
        Nombre de plis pour la cross-validation stratifiee
    tags : dict, optional
        Tags supplementaires pour le run MLflow
    description : str, optional
        Description textuelle du run
    optimize_threshold_flag : bool, default=True
        Active/desactive l'optimisation du seuil
    cost_fn : int, default=10
        Cout d'un Faux Negatif
    cost_fp : int, default=1
        Cout d'un Faux Positif
    threshold_range : tuple, default=(0.05, 0.95)
        Range des seuils a tester
    threshold_step : float, default=0.01
        Pas entre chaque seuil teste
    
    Returns:
    --------
    dict : Dictionnaire avec toutes les metriques et le run_id
    
    Example:
    --------
    >>> from sklearn.ensemble import RandomForestClassifier
    >>> model = RandomForestClassifier(class_weight='balanced')
    >>> results = train_and_log_model(
    ...     model, "RandomForest", X_train, y_train, X_val, y_val
    ... )
    """
    
    # ========================================
    # DEBUT DU RUN MLFLOW
    # ========================================
    
    with mlflow.start_run(run_name=model_name) as run:
        
        import os
        os.makedirs("graphiques", exist_ok=True)
        print(f"\n{'='*80}")
        print(f"MODELE : {model_name}")
        print(f"{'='*80}")
        print(f"Run ID : {run.info.run_id}")
        
        if description:
            mlflow.set_tag("mlflow.note.content", description)
        
        start_time = time.time()
        
        # ---------- LOG PARAMETRES ----------
        params = model.get_params()
        for param_name, param_value in params.items():
            if isinstance(param_value, (list, dict, tuple)):
                param_value = str(param_value)
            mlflow.log_param(param_name, param_value)
        
        mlflow.log_param("n_features", X_train.shape[1])
        mlflow.log_param("n_train_samples", X_train.shape[0])
        mlflow.log_param("n_val_samples", X_val.shape[0])
        mlflow.log_param("cv_folds", cv_folds)
        mlflow.log_param("cv_strategy", "StratifiedKFold")
        mlflow.log_param("cost_fn", cost_fn)
        mlflow.log_param("cost_fp", cost_fp)
        mlflow.log_param("optimize_threshold", optimize_threshold_flag)
        
        # ---------- CROSS-VALIDATION STRATIFIEE ----------
        print(f"\nCROSS-VALIDATION STRATIFIEE ({cv_folds} folds)")
        print(f"{'─'*80}")
        
        skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        
        scoring = {
            'roc_auc': 'roc_auc',
            'accuracy': 'accuracy',
            'precision': 'precision',
            'recall': 'recall',
            'f1': 'f1'
        }
        
        cv_results = cross_validate(
            model, X_train, y_train, 
            cv=skf, scoring=scoring,
            return_train_score=False, n_jobs=-1
        )
        
        # Calcul moyennes et ecarts-types
        cv_auc_mean = cv_results['test_roc_auc'].mean()
        cv_auc_std = cv_results['test_roc_auc'].std()
        cv_accuracy_mean = cv_results['test_accuracy'].mean()
        cv_accuracy_std = cv_results['test_accuracy'].std()
        cv_precision_mean = cv_results['test_precision'].mean()
        cv_precision_std = cv_results['test_precision'].std()
        cv_recall_mean = cv_results['test_recall'].mean()
        cv_recall_std = cv_results['test_recall'].std()
        cv_f1_mean = cv_results['test_f1'].mean()
        cv_f1_std = cv_results['test_f1'].std()
        
        # Log metriques CV
        mlflow.log_metric("cv_auc_roc_mean", cv_auc_mean)
        mlflow.log_metric("cv_auc_roc_std", cv_auc_std)
        mlflow.log_metric("cv_accuracy_mean", cv_accuracy_mean)
        mlflow.log_metric("cv_accuracy_std", cv_accuracy_std)
        mlflow.log_metric("cv_precision_mean", cv_precision_mean)
        mlflow.log_metric("cv_precision_std", cv_precision_std)
        mlflow.log_metric("cv_recall_mean", cv_recall_mean)
        mlflow.log_metric("cv_recall_std", cv_recall_std)
        mlflow.log_metric("cv_f1_mean", cv_f1_mean)
        mlflow.log_metric("cv_f1_std", cv_f1_std)
        
        # Affichage complet des metriques CV
        print(f"Resultats Cross-Validation ({cv_folds} folds) :")
        print(f"   - AUC-ROC   : {cv_auc_mean:.4f} (+/-{cv_auc_std:.4f})")
        print(f"   - Accuracy  : {cv_accuracy_mean:.4f} (+/-{cv_accuracy_std:.4f})")
        print(f"   - Precision : {cv_precision_mean:.4f} (+/-{cv_precision_std:.4f})")
        print(f"   - Recall    : {cv_recall_mean:.4f} (+/-{cv_recall_std:.4f})")
        print(f"   - F1-Score  : {cv_f1_mean:.4f} (+/-{cv_f1_std:.4f})")
        
        # ---------- ENTRAINEMENT FINAL ----------
        print(f"\nENTRAINEMENT FINAL")
        print(f"{'─'*80}")
        print(f"Entrainement sur {X_train.shape[0]:,} echantillons...")
        
        model.fit(X_train, y_train)
        training_time = time.time() - start_time
        mlflow.log_metric("training_time_seconds", training_time)
        
        print(f"Termine en {training_time:.2f}s")
        
        # ---------- PREDICTIONS VALIDATION ----------
        print(f"\nEVALUATION SUR VALIDATION SET")
        print(f"{'─'*80}")
        
        y_pred = model.predict(X_val)
        y_proba = model.predict_proba(X_val)[:, 1]
        
        # Metriques techniques
        auc = roc_auc_score(y_val, y_proba)
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred, zero_division=0)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred, zero_division=0)
        
        # Metriques classe 0
        precision_0 = precision_score(y_val, y_pred, pos_label=0, zero_division=0)
        recall_0 = recall_score(y_val, y_pred, pos_label=0)
        f1_0 = f1_score(y_val, y_pred, pos_label=0, zero_division=0)
        
        # Confusion matrix
        cm = confusion_matrix(y_val, y_pred)
        tn, fp, fn, tp = cm.ravel()
        
        # Log metriques validation
        mlflow.log_metric("val_auc_roc", auc)
        mlflow.log_metric("val_accuracy", accuracy)
        mlflow.log_metric("val_precision_class_1", precision)
        mlflow.log_metric("val_recall_class_1", recall)
        mlflow.log_metric("val_f1_score_class_1", f1)
        mlflow.log_metric("val_precision_class_0", precision_0)
        mlflow.log_metric("val_recall_class_0", recall_0)
        mlflow.log_metric("val_f1_score_class_0", f1_0)
        mlflow.log_metric("val_true_negatives", int(tn))
        mlflow.log_metric("val_false_positives", int(fp))
        mlflow.log_metric("val_false_negatives", int(fn))
        mlflow.log_metric("val_true_positives", int(tp))
        
        # Affichage complet des metriques validation
        print(f"\nMetriques Validation (Classe 1: Defaut) :")
        print(f"   - AUC-ROC   : {auc:.4f}")
        print(f"   - Accuracy  : {accuracy:.4f}")
        print(f"   - Precision : {precision:.4f}")
        print(f"   - Recall    : {recall:.4f}")
        print(f"   - F1-Score  : {f1:.4f}")
        
        print(f"\n   Metriques Classe 0 (Pas de Defaut) :")
        print(f"   - Precision : {precision_0:.4f}")
        print(f"   - Recall    : {recall_0:.4f}")
        print(f"   - F1-Score  : {f1_0:.4f}")
        
        print(f"\nMatrice de Confusion :")
        print(f"   - TN (Vrai Negatif)  : {tn:,}")
        print(f"   - FP (Faux Positif)  : {fp:,}")
        print(f"   - FN (Faux Negatif)  : {fn:,}")
        print(f"   - TP (Vrai Positif)  : {tp:,}")
        
        # ---------- COUT METIER SEUIL PAR DEFAUT ----------
        print(f"\nCOUT METIER (Seuil par defaut = 0.5)")
        print(f"{'─'*80}")
        
        cost_default = calculate_business_cost(y_val, y_pred, cost_fn, cost_fp)
        
        mlflow.log_metric("cost_at_default_threshold", cost_default['total_business_cost'])
        mlflow.log_metric("cost_fn_default", cost_default['cost_fn_total'])
        mlflow.log_metric("cost_fp_default", cost_default['cost_fp_total'])
        mlflow.log_metric("cost_per_prediction_default", cost_default['cost_per_prediction'])
        
        print(f"   - Cout FN (defauts manques)      : {cost_default['cost_fn_total']:,} ({cost_default['fn_count']} x {cost_fn})")
        print(f"   - Cout FP (bons clients refuses) : {cost_default['cost_fp_total']:,} ({cost_default['fp_count']} x {cost_fp})")
        print(f"   - COUT TOTAL                     : {cost_default['total_business_cost']:,}")
        
        # ---------- OPTIMISATION DU SEUIL ----------
        optimal_results = None
        optimization_time = 0
        
        if optimize_threshold_flag:
            print(f"\nOPTIMISATION DU SEUIL DE DECISION")
            print(f"{'─'*80}")
            
            optimization_start = time.time()
            n_thresholds = int((threshold_range[1] - threshold_range[0]) / threshold_step) + 1
            print(f"   Recherche du seuil optimal ({n_thresholds} seuils testes)...")
            
            optimal_results = optimize_threshold(y_val, y_proba, cost_fn, cost_fp, threshold_range, threshold_step)
            optimization_time = time.time() - optimization_start
            
            mlflow.log_metric("threshold_optimization_time", optimization_time)
            mlflow.log_metric("optimal_threshold", optimal_results['optimal_threshold'])
            mlflow.log_metric("cost_at_optimal_threshold", optimal_results['cost_at_optimal'])
            mlflow.log_metric("cost_fn_optimal", optimal_results['cost_fn_optimal'])
            mlflow.log_metric("cost_fp_optimal", optimal_results['cost_fp_optimal'])
            mlflow.log_metric("recall_at_optimal", optimal_results['recall_at_optimal'])
            mlflow.log_metric("precision_at_optimal", optimal_results['precision_at_optimal'])
            mlflow.log_metric("f1_at_optimal", optimal_results['f1_at_optimal'])
            mlflow.log_metric("fn_at_optimal", optimal_results['fn_at_optimal'])
            mlflow.log_metric("fp_at_optimal", optimal_results['fp_at_optimal'])
            mlflow.log_metric("tn_at_optimal", optimal_results['tn_at_optimal'])
            mlflow.log_metric("tp_at_optimal", optimal_results['tp_at_optimal'])
            
            cost_reduction = cost_default['total_business_cost'] - optimal_results['cost_at_optimal']
            cost_reduction_pct = (cost_reduction / cost_default['total_business_cost']) * 100
            mlflow.log_metric("cost_reduction_absolute", cost_reduction)
            mlflow.log_metric("cost_reduction_percent", cost_reduction_pct)
            
            print(f"   Seuil optimal trouve : {optimal_results['optimal_threshold']:.2f}")
            print(f"\n   Cout au seuil optimal :")
            print(f"      - Cout FN : {optimal_results['cost_fn_optimal']:,} ({optimal_results['fn_at_optimal']} x {cost_fn})")
            print(f"      - Cout FP : {optimal_results['cost_fp_optimal']:,} ({optimal_results['fp_at_optimal']} x {cost_fp})")
            print(f"      - COUT TOTAL : {optimal_results['cost_at_optimal']:,}")
            print(f"      - Economie : {cost_reduction:,} (-{cost_reduction_pct:.1f}%)")
            
            print(f"\n   Metriques au seuil optimal :")
            print(f"      - Recall    : {optimal_results['recall_at_optimal']:.4f}")
            print(f"      - Precision : {optimal_results['precision_at_optimal']:.4f}")
            print(f"      - F1-Score  : {optimal_results['f1_at_optimal']:.4f}")
        
        # ---------- ARTEFACTS ----------
        print(f"\nGENERATION DES VISUALISATIONS")
        print(f"{'─'*80}")
        
        # Confusion Matrix (seuil par defaut)
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Pas Defaut (0)', 'Defaut (1)'],
                    yticklabels=['Pas Defaut (0)', 'Defaut (1)'],
                    cbar_kws={'label': 'Nombre de predictions'})
        ax.set_title(f'Matrice de Confusion - {model_name}\nSeuil = 0.5 | AUC: {auc:.4f}',
                    fontsize=14, fontweight='bold')
        ax.set_ylabel('Vraie Classe', fontsize=12)
        ax.set_xlabel('Classe Predite', fontsize=12)
        plt.tight_layout()
        cm_path = f"graphiques/confusion_matrix_default_{model_name.replace(' ', '_')}.png"
        plt.savefig(cm_path, dpi=100, bbox_inches='tight')
        mlflow.log_artifact(cm_path)
        plt.close()
        print(f"   Matrice de confusion sauvegardee (seuil 0.5)")
        
        # Confusion Matrix (seuil optimal)
        if optimize_threshold_flag and optimal_results:
            cm_optimal = np.array([[optimal_results['tn_at_optimal'], optimal_results['fp_at_optimal']],
                                   [optimal_results['fn_at_optimal'], optimal_results['tp_at_optimal']]])
            
            fig, ax = plt.subplots(figsize=(8, 6))
            sns.heatmap(cm_optimal, annot=True, fmt='d', cmap='Greens',
                        xticklabels=['Pas Defaut (0)', 'Defaut (1)'],
                        yticklabels=['Pas Defaut (0)', 'Defaut (1)'],
                        cbar_kws={'label': 'Nombre de predictions'})
            ax.set_title(f'Matrice de Confusion - {model_name}\nSeuil optimal = {optimal_results["optimal_threshold"]:.2f} | Cout: {optimal_results["cost_at_optimal"]:,}',
                        fontsize=14, fontweight='bold')
            ax.set_ylabel('Vraie Classe', fontsize=12)
            ax.set_xlabel('Classe Predite', fontsize=12)
            plt.tight_layout()
            cm_opt_path = f"graphiques/confusion_matrix_optimal_{model_name.replace(' ', '_')}.png"
            plt.savefig(cm_opt_path, dpi=100, bbox_inches='tight')
            mlflow.log_artifact(cm_opt_path)
            plt.close()
            print(f"   Matrice de confusion sauvegardee (seuil optimal)")
        
        # ROC Curve
        fpr, tpr, _ = roc_curve(y_val, y_proba)
        
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.plot(fpr, tpr, label=f'Validation (AUC = {auc:.4f})',
               linewidth=2, color='#3498db')
        ax.plot([0, 1], [0, 1], 'k--', label='Baseline (AUC = 0.50)', linewidth=1)
        ax.fill_between([0, 1], 
                        [cv_auc_mean - cv_auc_std]*2, 
                        [cv_auc_mean + cv_auc_std]*2,
                        alpha=0.2, color='#3498db', 
                        label=f'CV AUC: {cv_auc_mean:.4f}+/-{cv_auc_std:.4f}')
        ax.set_xlabel('Taux de Faux Positifs (FPR)', fontsize=12)
        ax.set_ylabel('Taux de Vrais Positifs (TPR)', fontsize=12)
        ax.set_title(f'Courbe ROC - {model_name}', fontsize=14, fontweight='bold')
        ax.legend(loc='lower right', fontsize=10)
        ax.grid(alpha=0.3)
        plt.tight_layout()
        roc_path = f"graphiques/roc_curve_{model_name.replace(' ', '_')}.png"
        plt.savefig(roc_path, dpi=100, bbox_inches='tight')
        mlflow.log_artifact(roc_path)
        plt.close()
        print(f"   Courbe ROC sauvegardee")
        
        # Precision-Recall Curve
        precision_curve, recall_curve, _ = precision_recall_curve(y_val, y_proba)
        
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.plot(recall_curve, precision_curve, linewidth=2, color='#e74c3c')
        ax.axhline(y=precision, color='orange', linestyle='--', 
                  label=f'Precision (seuil 0.5): {precision:.3f}', alpha=0.7)
        ax.axvline(x=recall, color='green', linestyle='--', 
                  label=f'Recall (seuil 0.5): {recall:.3f}', alpha=0.7)
        ax.set_xlabel('Recall', fontsize=12)
        ax.set_ylabel('Precision', fontsize=12)
        ax.set_title(f'Courbe Precision-Recall - {model_name}',
                    fontsize=14, fontweight='bold')
        ax.legend(fontsize=10)
        ax.grid(alpha=0.3)
        plt.tight_layout()
        pr_path = f"graphiques/precision_recall_{model_name.replace(' ', '_')}.png"
        plt.savefig(pr_path, dpi=100, bbox_inches='tight')
        mlflow.log_artifact(pr_path)
        plt.close()
        print(f"   Courbe Precision-Recall sauvegardee")
        
        # Courbe Cout vs Seuil
        if optimize_threshold_flag and optimal_results:
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(optimal_results['all_thresholds'], optimal_results['all_costs'], 
                   linewidth=2, color='#e74c3c', label='Cout total')
            ax.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5, 
                      label=f'Seuil defaut (0.5) - Cout: {cost_default["total_business_cost"]:,}')
            ax.axvline(x=optimal_results['optimal_threshold'], color='green', linestyle='--', 
                      linewidth=2, label=f'Seuil optimal ({optimal_results["optimal_threshold"]:.2f}) - Cout: {optimal_results["cost_at_optimal"]:,}')
            ax.scatter([optimal_results['optimal_threshold']], [optimal_results['cost_at_optimal']], 
                      color='green', s=200, zorder=5, marker='*')
            ax.set_xlabel('Seuil de decision', fontsize=12)
            ax.set_ylabel('Cout metier total', fontsize=12)
            ax.set_title(f'Optimisation du Seuil - {model_name}\nCout FN = {cost_fn}x | Cout FP = {cost_fp}x', 
                        fontsize=14, fontweight='bold')
            ax.legend(fontsize=10)
            ax.grid(alpha=0.3)
            plt.tight_layout()
            cost_path = f"graphiques/cost_vs_threshold_{model_name.replace(' ', '_')}.png"
            plt.savefig(cost_path, dpi=100, bbox_inches='tight')
            mlflow.log_artifact(cost_path)
            plt.close()
            print(f"   Courbe Cout vs Seuil sauvegardee")
            
            # 6. Multi-metriques vs Seuil
            fig, axes = plt.subplots(3, 1, figsize=(12, 10))
            
            axes[0].plot(optimal_results['all_thresholds'], optimal_results['all_costs'], 
                        linewidth=2, color='#e74c3c')
            axes[0].axvline(x=optimal_results['optimal_threshold'], color='green', 
                           linestyle='--', alpha=0.7)
            axes[0].set_ylabel('Cout total', fontsize=11)
            axes[0].set_title('Cout metier vs Seuil', fontsize=12, fontweight='bold')
            axes[0].grid(alpha=0.3)
            
            axes[1].plot(optimal_results['all_thresholds'], optimal_results['all_recalls'], 
                        linewidth=2, color='#3498db', label='Recall')
            axes[1].axvline(x=optimal_results['optimal_threshold'], color='green', 
                           linestyle='--', alpha=0.7)
            axes[1].axhline(y=optimal_results['recall_at_optimal'], color='green', 
                           linestyle=':', alpha=0.5)
            axes[1].set_ylabel('Recall (Classe 1)', fontsize=11)
            axes[1].set_title('Recall vs Seuil', fontsize=12, fontweight='bold')
            axes[1].grid(alpha=0.3)
        
            axes[2].plot(optimal_results['all_thresholds'], optimal_results['all_precisions'], 
                        linewidth=2, color='#f39c12', label='Precision')
            axes[2].axvline(x=optimal_results['optimal_threshold'], color='green', 
                           linestyle='--', alpha=0.7)
            axes[2].axhline(y=optimal_results['precision_at_optimal'], color='green', 
                           linestyle=':', alpha=0.5)
            axes[2].set_xlabel('Seuil de decision', fontsize=11)
            axes[2].set_ylabel('Precision (Classe 1)', fontsize=11)
            axes[2].set_title('Precision vs Seuil', fontsize=12, fontweight='bold')
            axes[2].grid(alpha=0.3)
            
            plt.suptitle(f'Evolution des Metriques - {model_name}', fontsize=14, fontweight='bold')
            plt.tight_layout()
            metrics_path = f"graphiques/metrics_vs_threshold_{model_name.replace(' ', '_')}.png"
            plt.savefig(metrics_path, dpi=100, bbox_inches='tight')
            mlflow.log_artifact(metrics_path)
            plt.close()
            print(f"   Courbe Multi-metriques sauvegardee")
        
        # CV Scores Distribution
        fig, axes = plt.subplots(2, 3, figsize=(16, 10))
        axes = axes.ravel()
        
        metrics_cv = [
            ('AUC-ROC', cv_results['test_roc_auc'], cv_auc_mean, auc, '#3498db'),
            ('Accuracy', cv_results['test_accuracy'], cv_accuracy_mean, accuracy, '#9b59b6'),
            ('Precision', cv_results['test_precision'], cv_precision_mean, precision, '#2ecc71'),
            ('Recall', cv_results['test_recall'], cv_recall_mean, recall, '#e74c3c'),
            ('F1-Score', cv_results['test_f1'], cv_f1_mean, f1, '#f39c12')
        ]
        
        for idx, (metric_name, scores, mean_val, val_score, color) in enumerate(metrics_cv):
            axes[idx].bar(range(cv_folds), scores, color=color, alpha=0.7)
            axes[idx].axhline(y=mean_val, color='red', linestyle='--', 
                            label=f'CV Mean: {mean_val:.4f}', linewidth=2)
            axes[idx].axhline(y=val_score, color='green', linestyle='--', 
                            label=f'Val: {val_score:.4f}', linewidth=2)
            axes[idx].set_xlabel('Fold', fontsize=10)
            axes[idx].set_ylabel(metric_name, fontsize=10)
            axes[idx].set_title(f'{metric_name} par Fold', fontsize=12, fontweight='bold')
            axes[idx].legend(fontsize=9)
            axes[idx].grid(alpha=0.3)
        
        # Dernier subplot : Resume
        axes[5].axis('off')
        summary_text = f"""
        RESUME CROSS-VALIDATION
        
        Strategie: StratifiedKFold ({cv_folds} folds)
        
        AUC-ROC  : {cv_auc_mean:.4f} +/- {cv_auc_std:.4f}
        Accuracy : {cv_accuracy_mean:.4f} +/- {cv_accuracy_std:.4f}
        Precision: {cv_precision_mean:.4f} +/- {cv_precision_std:.4f}
        Recall   : {cv_recall_mean:.4f} +/- {cv_recall_std:.4f}
        F1-Score : {cv_f1_mean:.4f} +/- {cv_f1_std:.4f}
        """
        axes[5].text(0.1, 0.5, summary_text, fontsize=11, family='monospace',
                    verticalalignment='center')
        
        plt.suptitle(f'Cross-Validation Scores - {model_name}', 
                    fontsize=16, fontweight='bold', y=0.995)
        plt.tight_layout()
        cv_path = f"graphiques/cv_scores_{model_name.replace(' ', '_')}.png"
        plt.savefig(cv_path, dpi=100, bbox_inches='tight')
        mlflow.log_artifact(cv_path)
        plt.close()
        print(f"   Scores CV sauvegardes")
        
        # ---------- SAUVEGARDER MODELE ----------
        print(f"\nSAUVEGARDE DU MODELE")
        print(f"{'─'*80}")
        
        if hasattr(X_train, 'head'):
            input_example = X_train.head(5)
        else:
            input_example = X_train[:5]
        
        import sklearn
        
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            input_example=input_example,
            pip_requirements=[
                f"scikit-learn=={sklearn.__version__}",
                f"numpy=={np.__version__}",
                f"pandas=={pd.__version__}"
            ]
        )
        print(f"   Modele enregistre dans MLflow")
        
        # ---------- TAGS ----------
        default_tags = {
            "model_family": type(model).__name__,
            "validation_strategy": f"StratifiedKFold_{cv_folds}",
            "feature_engineering": "complete",
            "cost_optimized": str(optimize_threshold_flag)
        }
        
        if optimize_threshold_flag and optimal_results:
            default_tags["optimal_threshold_found"] = f"{optimal_results['optimal_threshold']:.2f}"
            default_tags["cost_reduction_percent"] = f"{cost_reduction_pct:.1f}%"
        
        if tags:
            default_tags.update(tags)
        mlflow.set_tags(default_tags)
        
        # ---------- RESUME FINAL ----------
        print(f"\n{'='*80}")
        print(f"RESULTATS FINAUX - {model_name}")
        print(f"{'='*80}")
        print(f"Temps d'entrainement : {training_time:.2f}s")
        if optimize_threshold_flag:
            print(f"Temps optimisation seuil : {optimization_time:.2f}s")
        
        print(f"\nCROSS-VALIDATION ({cv_folds} folds stratifies) :")
        print(f"   - AUC-ROC   : {cv_auc_mean:.4f} (+/-{cv_auc_std:.4f})")
        print(f"   - Accuracy  : {cv_accuracy_mean:.4f} (+/-{cv_accuracy_std:.4f})")
        print(f"   - Precision : {cv_precision_mean:.4f} (+/-{cv_precision_std:.4f})")
        print(f"   - Recall    : {cv_recall_mean:.4f} (+/-{cv_recall_std:.4f})")
        print(f"   - F1-Score  : {cv_f1_mean:.4f} (+/-{cv_f1_std:.4f})")
        
        print(f"\nVALIDATION SET (Seuil 0.5) :")
        print(f"   - AUC-ROC   : {auc:.4f}")
        print(f"   - Accuracy  : {accuracy:.4f}")
        print(f"   - Precision : {precision:.4f}")
        print(f"   - Recall    : {recall:.4f}")
        print(f"   - F1-Score  : {f1:.4f}")
        print(f"   - TN/FP/FN/TP : {tn:,} / {fp:,} / {fn:,} / {tp:,}")
        print(f"   - Cout metier : {cost_default['total_business_cost']:,}")
        
        if optimize_threshold_flag and optimal_results:
            print(f"\nCOUT METIER OPTIMISE :")
            print(f"   - Seuil optimal          : {optimal_results['optimal_threshold']:.2f}")
            print(f"   - Cout au seuil optimal  : {optimal_results['cost_at_optimal']:,} (vs {cost_default['total_business_cost']:,} au seuil 0.5)")
            print(f"   - Economie realisee      : {cost_reduction:,} (-{cost_reduction_pct:.1f}%)")
            print(f"   - Recall au seuil optimal: {optimal_results['recall_at_optimal']:.4f}")
            print(f"   - Defauts detectes       : {optimal_results['tp_at_optimal']:,} (vs {tp:,})")
            print(f"   - Defauts manques        : {optimal_results['fn_at_optimal']:,} (vs {fn:,})")
        
        print(f"{'='*80}")
        print(f"Run termine - ID: {run.info.run_id}")
        print(f"{'='*80}\n")
        
        # Retourner metriques
        result_dict = {
            "run_id": run.info.run_id,
            "model_name": model_name,
            # CV metrics
            "cv_auc_roc_mean": cv_auc_mean,
            "cv_auc_roc_std": cv_auc_std,
            "cv_accuracy_mean": cv_accuracy_mean,
            "cv_precision_mean": cv_precision_mean,
            "cv_recall_mean": cv_recall_mean,
            "cv_f1_mean": cv_f1_mean,
            # Validation metrics (seuil par defaut)
            "val_auc_roc": auc,
            "val_accuracy": accuracy,
            "val_precision": precision,
            "val_recall": recall,
            "val_f1": f1,
            "val_tn": int(tn),
            "val_fp": int(fp),
            "val_fn": int(fn),
            "val_tp": int(tp),
            # Cout metier seuil par defaut
            "cost_at_default_threshold": cost_default['total_business_cost'],
            "cost_fn_default": cost_default['cost_fn_total'],
            "cost_fp_default": cost_default['cost_fp_total'],
            # Temps
            "training_time": training_time
        }
        
        # Ajouter metriques optimisation si activee
        if optimize_threshold_flag and optimal_results:
            result_dict.update({
                "optimal_threshold": optimal_results['optimal_threshold'],
                "cost_at_optimal_threshold": optimal_results['cost_at_optimal'],
                "cost_reduction_absolute": cost_reduction,
                "cost_reduction_percent": cost_reduction_pct,
                "cost_fn_optimal": optimal_results['cost_fn_optimal'],
                "cost_fp_optimal": optimal_results['cost_fp_optimal'],
                "recall_at_optimal": optimal_results['recall_at_optimal'],
                "precision_at_optimal": optimal_results['precision_at_optimal'],
                "f1_at_optimal": optimal_results['f1_at_optimal'],
                "fn_at_optimal": optimal_results['fn_at_optimal'],
                "fp_at_optimal": optimal_results['fp_at_optimal'],
                "tn_at_optimal": optimal_results['tn_at_optimal'],
                "tp_at_optimal": optimal_results['tp_at_optimal'],
                "threshold_optimization_time": optimization_time
            })
        
        return result_dict

print("Fonction train_and_log_model() avec optimisation du cout metier creee")

Fonction train_and_log_model() avec optimisation du cout metier creee


<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Régression logistique</h3>

In [11]:
# NORMALISATION POUR REGRESSION LOGISTIQUE

print("NORMALISATION DES DONNÉES POUR LOGISTIC REGRESSION")
print("="*70)

# Créer le scaler
scaler = StandardScaler()

# Fit sur TRAIN uniquement (éviter data leakage)
print("Fit StandardScaler sur train...")
X_train_scaled = scaler.fit_transform(X_train_encoded)

# Transform validation
print("Transform validation...")
X_val_scaled = scaler.transform(X_val_encoded)

# Convertir en DataFrame pour garder cohérence
X_train_scaled = pd.DataFrame(
    X_train_scaled,
    columns=X_train_encoded.columns,
    index=X_train_encoded.index
)

X_val_scaled = pd.DataFrame(
    X_val_scaled,
    columns=X_val_encoded.columns,
    index=X_val_encoded.index
)

print(f"Normalisation terminée")
print(f"Train scaled: {X_train_scaled.shape}")
print(f"Val scaled: {X_val_scaled.shape}")
print()

# ========================================
# LOGISTIC REGRESSION AVEC DONNÉES NORMALISÉES
# ========================================

mlflow.set_experiment("home_credit_scoring")

model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42,
    solver='saga',      # grands datasets
    penalty='l2',       # Régularisation L2
    C=1.0,              # Force de régularisation
    n_jobs=-1           # Parallélisation
)

results = train_and_log_model(
    model=model,
    model_name="LogisticRegression_Scaled",
    X_train=X_train_scaled,
    y_train=y_train,
    X_val=X_val_scaled,
    y_val=y_val,
    cv_folds=5,
    description="Logistic Regression avec StandardScaler + class_weight balanced",
    tags={"preprocessing": "StandardScaler", "type": "hyperparameters_default"}
)

# Résultats
print(f"\nCOMPARAISON AVANT/APRÈS NORMALISATION")
print(f"Avant (sans scaling) : AUC = 0.6580")
print(f"Après (avec scaling) : AUC = {results['val_auc_roc']:.4f}")

NORMALISATION DES DONNÉES POUR LOGISTIC REGRESSION
Fit StandardScaler sur train...
Transform validation...
Normalisation terminée
Train scaled: (246008, 912)
Val scaled: (61503, 912)


MODELE : LogisticRegression_Scaled
Run ID : b98d31da41db454aa9510848c94ceffb

CROSS-VALIDATION STRATIFIEE (5 folds)
────────────────────────────────────────────────────────────────────────────────
Resultats Cross-Validation (5 folds) :
   - AUC-ROC   : 0.7696 (+/-0.0032)
   - Accuracy  : 0.7061 (+/-0.0020)
   - Precision : 0.1730 (+/-0.0015)
   - Recall    : 0.6984 (+/-0.0050)
   - F1-Score  : 0.2773 (+/-0.0023)

ENTRAINEMENT FINAL
────────────────────────────────────────────────────────────────────────────────
Entrainement sur 246,008 echantillons...
Termine en 1300.44s

EVALUATION SUR VALIDATION SET
────────────────────────────────────────────────────────────────────────────────

Metriques Validation (Classe 1: Defaut) :
   - AUC-ROC   : 0.7739
   - Accuracy  : 0.7087
   - Precision : 0.1742
   - Recal

2025/10/09 12:12:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 12:12:57 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


   Scores CV sauvegardes

SAUVEGARDE DU MODELE
────────────────────────────────────────────────────────────────────────────────
   Modele enregistre dans MLflow

RESULTATS FINAUX - LogisticRegression_Scaled
Temps d'entrainement : 1300.44s
Temps optimisation seuil : 0.80s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7696 (+/-0.0032)
   - Accuracy  : 0.7061 (+/-0.0020)
   - Precision : 0.1730 (+/-0.0015)
   - Recall    : 0.6984 (+/-0.0050)
   - F1-Score  : 0.2773 (+/-0.0023)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7739
   - Accuracy  : 0.7087
   - Precision : 0.1742
   - Recall    : 0.6975
   - F1-Score  : 0.2788
   - TN/FP/FN/TP : 40,122 / 16,416 / 1,502 / 3,463
   - Cout metier : 31,436

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.53
   - Cout au seuil optimal  : 31,408 (vs 31,436 au seuil 0.5)
   - Economie realisee      : 28 (-0.1%)
   - Recall au seuil optimal: 0.6568
   - Defauts detectes       : 3,261 (vs 3,463)
   - Defauts manques        : 1,70

<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">RANDOM FOREST</h3>

In [13]:
# Configuration
mlflow.set_experiment("home_credit_scoring")

# RANDOM Forest
model = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    max_depth=10,
    random_state=42
)

# Lancer l'entraînement
results = train_and_log_model(
    model=model,
    model_name="RandomForest_Balanced",
    X_train=X_train_encoded,
    y_train=y_train,
    X_val=X_val_encoded,
    y_val=y_val,
    cv_folds=5,
    description="Random Forest avec class_weight balanced",
    tags={"type": "hyperparameters_default"}
)

# Résultats
print(f"AUC CV: {results['cv_auc_roc_mean']:.4f}")
print(f"AUC Val: {results['val_auc_roc']:.4f}")


MODELE : RandomForest_Balanced
Run ID : 6e94b84436d6467996e1819079140725

CROSS-VALIDATION STRATIFIEE (5 folds)
────────────────────────────────────────────────────────────────────────────────
Resultats Cross-Validation (5 folds) :
   - AUC-ROC   : 0.7500 (+/-0.0028)
   - Accuracy  : 0.7624 (+/-0.0008)
   - Precision : 0.1863 (+/-0.0010)
   - Recall    : 0.5769 (+/-0.0073)
   - F1-Score  : 0.2816 (+/-0.0020)

ENTRAINEMENT FINAL
────────────────────────────────────────────────────────────────────────────────
Entrainement sur 246,008 echantillons...
Termine en 186.21s

EVALUATION SUR VALIDATION SET
────────────────────────────────────────────────────────────────────────────────

Metriques Validation (Classe 1: Defaut) :
   - AUC-ROC   : 0.7535
   - Accuracy  : 0.7565
   - Precision : 0.1869
   - Recall    : 0.6022
   - F1-Score  : 0.2853

   Metriques Classe 0 (Pas de Defaut) :
   - Precision : 0.9566
   - Recall    : 0.7700
   - F1-Score  : 0.8532

Matrice de Confusion :
   - TN (Vrai 

2025/10/09 12:17:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 12:17:15 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


   Scores CV sauvegardes

SAUVEGARDE DU MODELE
────────────────────────────────────────────────────────────────────────────────
   Modele enregistre dans MLflow

RESULTATS FINAUX - RandomForest_Balanced
Temps d'entrainement : 186.21s
Temps optimisation seuil : 0.79s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7500 (+/-0.0028)
   - Accuracy  : 0.7624 (+/-0.0008)
   - Precision : 0.1863 (+/-0.0010)
   - Recall    : 0.5769 (+/-0.0073)
   - F1-Score  : 0.2816 (+/-0.0020)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7535
   - Accuracy  : 0.7565
   - Precision : 0.1869
   - Recall    : 0.6022
   - F1-Score  : 0.2853
   - TN/FP/FN/TP : 43,534 / 13,004 / 1,975 / 2,990
   - Cout metier : 32,754

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.48
   - Cout au seuil optimal  : 32,671 (vs 32,754 au seuil 0.5)
   - Economie realisee      : 83 (-0.3%)
   - Recall au seuil optimal: 0.6475
   - Defauts detectes       : 3,215 (vs 2,990)
   - Defauts manques        : 1,750 (vs

<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">XGBoost</h3>

In [14]:
# PRÉPARATION DONNÉES POUR XGBOOST

print("NETTOYAGE DES NOMS DE COLONNES POUR XGBOOST")
print("="*70)

# Créer des copies pour XGBoost
X_train_xgb = X_train_encoded.copy()
X_val_xgb = X_val_encoded.copy()

# Nettoyer les noms de colonnes (enlever caractères spéciaux)
X_train_xgb.columns = X_train_xgb.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)
X_val_xgb.columns = X_val_xgb.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)

print(f"Noms de colonnes nettoyés pour XGBoost")
print(f"Exemple avant : BUREAU_CREDIT_ACTIVE_<LAMBDA_0>")
print(f"Exemple après : BUREAU_CREDIT_ACTIVE__LAMBDA_0_")
print()

# ========================================
# ENTRAÎNEMENT XGBOOST
# ========================================

# Configuration
mlflow.set_experiment("home_credit_scoring")

# Modèle XGBoost
model = XGBClassifier(
    n_estimators=150,
    scale_pos_weight=11.4,      # Ratio classes (91.9/8.1)
    learning_rate=0.05,
    max_depth=7,
    random_state=42,
    tree_method='hist',         # Plus rapide pour grands datasets
    eval_metric='auc',          # Métrique d'évaluation
    verbosity=0                 # Désactiver logs verbeux
)

# Lancer l'entraînement
results = train_and_log_model(
    model=model,
    model_name="XGBoost_Balanced",
    X_train=X_train_xgb,        # Utilisation des données nettoyées
    y_train=y_train,
    X_val=X_val_xgb,            # Utilisation des données nettoyées
    y_val=y_val,
    cv_folds=5,
    description="XGBoost avec scale_pos_weight=11.4 (ratio classes)",
    tags={"preprocessing": "column_names_cleaned", "type": "hyperparameters_default"}
)

# Résultats
print(f"\nRÉSULTATS XGBOOST")
print(f"AUC CV  : {results['cv_auc_roc_mean']:.4f}")
print(f"AUC Val : {results['val_auc_roc']:.4f}")

NETTOYAGE DES NOMS DE COLONNES POUR XGBOOST
Noms de colonnes nettoyés pour XGBoost
Exemple avant : BUREAU_CREDIT_ACTIVE_<LAMBDA_0>
Exemple après : BUREAU_CREDIT_ACTIVE__LAMBDA_0_


MODELE : XGBoost_Balanced
Run ID : 4c01e07f12cf46fc9aa38f6c7e89c83e

CROSS-VALIDATION STRATIFIEE (5 folds)
────────────────────────────────────────────────────────────────────────────────
Resultats Cross-Validation (5 folds) :
   - AUC-ROC   : 0.7735 (+/-0.0015)
   - Accuracy  : 0.7568 (+/-0.0014)
   - Precision : 0.1932 (+/-0.0017)
   - Recall    : 0.6337 (+/-0.0042)
   - F1-Score  : 0.2961 (+/-0.0024)

ENTRAINEMENT FINAL
────────────────────────────────────────────────────────────────────────────────
Entrainement sur 246,008 echantillons...
Termine en 57.93s

EVALUATION SUR VALIDATION SET
────────────────────────────────────────────────────────────────────────────────

Metriques Validation (Classe 1: Defaut) :
   - AUC-ROC   : 0.7772
   - Accuracy  : 0.7517
   - Precision : 0.1932
   - Recall    : 0.6538
 

2025/10/09 12:20:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 12:20:21 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


   Scores CV sauvegardes

SAUVEGARDE DU MODELE
────────────────────────────────────────────────────────────────────────────────
   Modele enregistre dans MLflow

RESULTATS FINAUX - XGBoost_Balanced
Temps d'entrainement : 57.93s
Temps optimisation seuil : 0.81s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7735 (+/-0.0015)
   - Accuracy  : 0.7568 (+/-0.0014)
   - Precision : 0.1932 (+/-0.0017)
   - Recall    : 0.6337 (+/-0.0042)
   - F1-Score  : 0.2961 (+/-0.0024)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7772
   - Accuracy  : 0.7517
   - Precision : 0.1932
   - Recall    : 0.6538
   - F1-Score  : 0.2983
   - TN/FP/FN/TP : 42,986 / 13,552 / 1,719 / 3,246
   - Cout metier : 30,742

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.50
   - Cout au seuil optimal  : 30,742 (vs 30,742 au seuil 0.5)
   - Economie realisee      : 0 (-0.0%)
   - Recall au seuil optimal: 0.6538
   - Defauts detectes       : 3,246 (vs 3,246)
   - Defauts manques        : 1,719 (vs 1,719)

<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">LightGBM </h3>

In [46]:
# PRÉPARATION DONNÉES POUR LIGHTGBM

# Nettoyer les noms de colonnes pour LightGBM (enlever caractères spéciaux)
X_train_lgbm = X_train_encoded.copy()
X_val_lgbm = X_val_encoded.copy()

# Remplacer les caractères spéciaux dans les noms de colonnes
X_train_lgbm.columns = X_train_lgbm.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)
X_val_lgbm.columns = X_val_lgbm.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)

print(f"Noms de colonnes nettoyés pour LightGBM")
print(f"Exemple avant: {X_train_encoded.columns[0]}")
print(f"Exemple après: {X_train_lgbm.columns[0]}")

# ========================================
# ENTRAÎNEMENT LIGHTGBM
# ========================================


# Configuration
mlflow.set_experiment("home_credit_scoring")

# Modèle
model = LGBMClassifier(
    n_estimators=150,
    class_weight='balanced',
    learning_rate=0.05,
    max_depth=7,
    random_state=42,
    verbose=-1,
    n_jobs=-1
)

# Lancer l'entraînement
results = train_and_log_model(
    model=model,
    model_name="LightGBM_Balanced",
    X_train=X_train_lgbm,
    y_train=y_train,
    X_val=X_val_lgbm,
    y_val=y_val,
    cv_folds=5,
    description="LightGBM avec class_weight balanced",
    tags={"preprocessing": "StandardScaler", "type": "hyperparameters_default"}
)

# Résultats
print(f"AUC CV: {results['cv_auc_roc_mean']:.4f}")
print(f"AUC Val: {results['val_auc_roc']:.4f}")

Noms de colonnes nettoyés pour LightGBM
Exemple avant: SK_ID_CURR
Exemple après: SK_ID_CURR

MODELE : LightGBM_Balanced
Run ID : f2af4e52a86c4eafb845ffa5ee6fc4fb

CROSS-VALIDATION STRATIFIEE (5 folds)
────────────────────────────────────────────────────────────────────────────────
Resultats Cross-Validation (5 folds) :
   - AUC-ROC   : 0.7764 (+/-0.0027)
   - Accuracy  : 0.7210 (+/-0.0020)
   - Precision : 0.1797 (+/-0.0024)
   - Recall    : 0.6889 (+/-0.0075)
   - F1-Score  : 0.2851 (+/-0.0036)

ENTRAINEMENT FINAL
────────────────────────────────────────────────────────────────────────────────
Entrainement sur 246,008 echantillons...
Termine en 35.39s

EVALUATION SUR VALIDATION SET
────────────────────────────────────────────────────────────────────────────────

Metriques Validation (Classe 1: Defaut) :
   - AUC-ROC   : 0.7795
   - Accuracy  : 0.7212
   - Precision : 0.1800
   - Recall    : 0.6902
   - F1-Score  : 0.2856

   Metriques Classe 0 (Pas de Defaut) :
   - Precision : 0.9638

2025/10/09 18:50:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 18:50:00 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


   Scores CV sauvegardes

SAUVEGARDE DU MODELE
────────────────────────────────────────────────────────────────────────────────
   Modele enregistre dans MLflow

RESULTATS FINAUX - LightGBM_Balanced
Temps d'entrainement : 35.39s
Temps optimisation seuil : 0.82s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7764 (+/-0.0027)
   - Accuracy  : 0.7210 (+/-0.0020)
   - Precision : 0.1797 (+/-0.0024)
   - Recall    : 0.6889 (+/-0.0075)
   - F1-Score  : 0.2851 (+/-0.0036)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7795
   - Accuracy  : 0.7212
   - Precision : 0.1800
   - Recall    : 0.6902
   - F1-Score  : 0.2856
   - TN/FP/FN/TP : 40,930 / 15,608 / 1,538 / 3,427
   - Cout metier : 30,988

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.52
   - Cout au seuil optimal  : 30,876 (vs 30,988 au seuil 0.5)
   - Economie realisee      : 112 (-0.4%)
   - Recall au seuil optimal: 0.6659
   - Defauts detectes       : 3,306 (vs 3,427)
   - Defauts manques        : 1,659 (vs 1,5

<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Comparaison des modèle baseline</h3>

In [54]:
# COMPARAISON DES MODÈLES BASELINE

print("="*80)
print("COMPARAISON DES MODÈLES BASELINE")
print("="*80)

# Connexion MLflow
client = MlflowClient()
experiment = client.get_experiment_by_name("home_credit_scoring")

# Récupérer tous les runs baseline
runs = client.search_runs(
    experiment.experiment_id,
    filter_string="tags.type = 'hyperparameters_default'",
    order_by=["metrics.cost_at_optimal_threshold ASC"],
    max_results=10
)

print(f"\nNombre de runs baseline trouvés : {len(runs)}")
print()

# ================================================================================
# EXTRACTION DES DONNÉES

comparison_data = []

for run in runs:
    model_name = run.data.tags.get("mlflow.runName", "Unknown")
    
    # Métriques clés
    metrics = {
        "Modèle": model_name,
        "Run ID": run.info.run_id[:8],
        
        # Coût métier
        "Coût (seuil 0.5)": run.data.metrics.get("cost_at_default_threshold"),
        "Seuil optimal": run.data.metrics.get("optimal_threshold"),
        "Coût optimal": run.data.metrics.get("cost_at_optimal_threshold"),
        "Économie": run.data.metrics.get("cost_reduction_absolute"),
        "Économie %": run.data.metrics.get("cost_reduction_percent"),
        
        # Métriques techniques
        "AUC-ROC": run.data.metrics.get("val_auc_roc"),
        "Accuracy": run.data.metrics.get("val_accuracy"),
        
        # Seuil par défaut
        "Recall (0.5)": run.data.metrics.get("val_recall_class_1"),
        "Precision (0.5)": run.data.metrics.get("val_precision_class_1"),
        "F1-Score (0.5)": run.data.metrics.get("val_f1_score_class_1"),
        
        # Seuil optimal
        "Recall (optimal)": run.data.metrics.get("recall_at_optimal"),
        "Precision (optimal)": run.data.metrics.get("precision_at_optimal"),
        "F1-Score (optimal)": run.data.metrics.get("f1_at_optimal"),
        
        # Confusion matrix seuil optimal
        "FN (optimal)": run.data.metrics.get("fn_at_optimal"),
        "FP (optimal)": run.data.metrics.get("fp_at_optimal"),
        "TP (optimal)": run.data.metrics.get("tp_at_optimal"),
        "TN (optimal)": run.data.metrics.get("tn_at_optimal"),
        
        # Temps
        "Temps (s)": run.data.metrics.get("training_time_seconds"),
        "Temps optim seuil (s)": run.data.metrics.get("threshold_optimization_time"),
        
        # Cross-validation
        "CV AUC": run.data.metrics.get("cv_auc_roc_mean"),
        "CV AUC std": run.data.metrics.get("cv_auc_roc_std"),
    }
    
    comparison_data.append(metrics)

# Créer DataFrame
df_comparison = pd.DataFrame(comparison_data)

# Trier par coût optimal
df_comparison = df_comparison.sort_values("Coût optimal", ascending=True)

print("="*80)
print("TABLEAU COMPARATIF COMPLET")
print("="*80)
print()

# ================================================================================
# AFFICHAGE DU COÛT MÉTIER 

print("1. COMPARAISON SUR LE COÛT MÉTIER (CRITÈRE PRINCIPAL)")
print("-"*80)

df_cost = df_comparison[[
    "Modèle", 
    "Coût (seuil 0.5)", 
    "Seuil optimal", 
    "Coût optimal", 
    "Économie",
    "Économie %"
]].copy()

# Formater pour affichage
df_cost["Coût (seuil 0.5)"] = df_cost["Coût (seuil 0.5)"].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else "N/A")
df_cost["Seuil optimal"] = df_cost["Seuil optimal"].apply(lambda x: f"{x:.2f}" if pd.notna(x) else "N/A")
df_cost["Coût optimal"] = df_cost["Coût optimal"].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else "N/A")
df_cost["Économie"] = df_cost["Économie"].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else "N/A")
df_cost["Économie %"] = df_cost["Économie %"].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "N/A")

print(df_cost.to_string(index=False))
print()

# ================================================================================
# AFFICHAGE : MÉTRIQUES TECHNIQUES

print("2. MÉTRIQUES TECHNIQUES (AUC, ACCURACY)")
print("-"*80)

df_tech = df_comparison[[
    "Modèle", 
    "AUC-ROC", 
    "CV AUC",
    "CV AUC std",
    "Accuracy",
    "Temps (s)"
]].copy()

# Formater
df_tech["AUC-ROC"] = df_tech["AUC-ROC"].apply(lambda x: f"{x:.4f}" if pd.notna(x) else "N/A")
df_tech["CV AUC"] = df_tech["CV AUC"].apply(lambda x: f"{x:.4f}" if pd.notna(x) else "N/A")
df_tech["CV AUC std"] = df_tech["CV AUC std"].apply(lambda x: f"{x:.4f}" if pd.notna(x) else "N/A")
df_tech["Accuracy"] = df_tech["Accuracy"].apply(lambda x: f"{x:.4f}" if pd.notna(x) else "N/A")
df_tech["Temps (s)"] = df_tech["Temps (s)"].apply(lambda x: f"{x:.1f}" if pd.notna(x) else "N/A")

print(df_tech.to_string(index=False))
print()

# ================================================================================
# AFFICHAGE : DÉTECTION DES DÉFAUTS

print("3. DÉTECTION DES DÉFAUTS (SEUIL OPTIMAL)")
print("-"*80)

df_defauts = df_comparison[[
    "Modèle",
    "Recall (optimal)",
    "Precision (optimal)",
    "F1-Score (optimal)",
    "TP (optimal)",
    "FN (optimal)",
    "FP (optimal)"
]].copy()

# Formater
df_defauts["Recall (optimal)"] = df_defauts["Recall (optimal)"].apply(lambda x: f"{x:.4f}" if pd.notna(x) else "N/A")
df_defauts["Precision (optimal)"] = df_defauts["Precision (optimal)"].apply(lambda x: f"{x:.4f}" if pd.notna(x) else "N/A")
df_defauts["F1-Score (optimal)"] = df_defauts["F1-Score (optimal)"].apply(lambda x: f"{x:.4f}" if pd.notna(x) else "N/A")
df_defauts["TP (optimal)"] = df_defauts["TP (optimal)"].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else "N/A")
df_defauts["FN (optimal)"] = df_defauts["FN (optimal)"].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else "N/A")
df_defauts["FP (optimal)"] = df_defauts["FP (optimal)"].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else "N/A")

print(df_defauts.to_string(index=False))
print()

# ================================================================================
# ANALYSE ET RECOMMANDATION

print("="*80)
print("ANALYSE ET RECOMMANDATION")
print("="*80)
print()

# Récupérer les données brutes (non formatées) pour l'analyse
best_cost_model = df_comparison.iloc[0]
best_auc_model = df_comparison.sort_values("AUC-ROC", ascending=False).iloc[0]
fastest_model = df_comparison.sort_values("Temps (s)", ascending=True).iloc[0]

print("MEILLEUR MODÈLE PAR CRITÈRE:")
print()
print(f"1. COÛT MÉTIER OPTIMAL (critère principal):")
print(f"   Modèle : {best_cost_model['Modèle']}")
print(f"   Coût optimal : {best_cost_model['Coût optimal']:,.0f}")
print(f"   Seuil optimal : {best_cost_model['Seuil optimal']:.2f}")
print(f"   Économie vs seuil 0.5 : {best_cost_model['Économie']:,.0f} (-{best_cost_model['Économie %']:.1f}%)")
print(f"   AUC-ROC : {best_cost_model['AUC-ROC']:.4f}")
print(f"   Recall (détection défauts) : {best_cost_model['Recall (optimal)']:.4f}")
print()

print(f"2. AUC-ROC (critère secondaire):")
print(f"   Modèle : {best_auc_model['Modèle']}")
print(f"   AUC-ROC : {best_auc_model['AUC-ROC']:.4f}")
print(f"   Coût optimal : {best_auc_model['Coût optimal']:,.0f}")
print()

print(f"3. TEMPS D'ENTRAÎNEMENT (critère tertiaire):")
print(f"   Modèle : {fastest_model['Modèle']}")
print(f"   Temps : {fastest_model['Temps (s)']:.1f}s")
print(f"   Coût optimal : {fastest_model['Coût optimal']:,.0f}")
print()

# ================================================================================
# RECOMMANDATION FINALE

print("="*80)
print("RECOMMANDATION POUR L'ÉTAPE 4 (OPTIMISATION HYPERPARAMÈTRES)")
print("="*80)
print()

# Vérifier si meilleur coût = meilleur AUC
if best_cost_model['Modèle'] == best_auc_model['Modèle']:
    print(f"RECOMMANDATION : {best_cost_model['Modèle']}")
    print()
    print("Ce modèle est optimal sur TOUS les critères :")
    print(f"   - Coût métier le plus bas : {best_cost_model['Coût optimal']:,.0f}")
    print(f"   - Meilleur AUC-ROC : {best_cost_model['AUC-ROC']:.4f}")
    print(f"   - Bon équilibre Recall/Precision")
    print()
    print(f"PROCHAINE ÉTAPE : Optimiser les hyperparamètres de {best_cost_model['Modèle']} avec Optuna")
else:
    print("ANALYSE : Compromis entre coût et AUC")
    print()
    print(f"Option A (recommandée) : {best_cost_model['Modèle']}")
    print(f"   - Meilleur coût métier : {best_cost_model['Coût optimal']:,.0f}")
    print(f"   - AUC : {best_cost_model['AUC-ROC']:.4f}")
    print(f"   - Avantage : Minimise le risque financier (objectif principal)")
    print()
    print(f"Option B : {best_auc_model['Modèle']}")
    print(f"   - Meilleur AUC : {best_auc_model['AUC-ROC']:.4f}")
    print(f"   - Coût : {best_auc_model['Coût optimal']:,.0f}")
    print(f"   - Différence de coût : +{best_auc_model['Coût optimal'] - best_cost_model['Coût optimal']:,.0f}")
    print()
    
    # Calculer l'écart
    cost_diff_pct = ((best_auc_model['Coût optimal'] - best_cost_model['Coût optimal']) / best_cost_model['Coût optimal']) * 100
    
    if cost_diff_pct < 2:
        print(f"Différence de coût négligeable ({cost_diff_pct:.1f}%)")
        print(f"RECOMMANDATION : Optimiser {best_auc_model['Modèle']} (meilleur AUC)")
    else:
        print(f"Différence de coût significative ({cost_diff_pct:.1f}%)")
        print(f"RECOMMANDATION : Optimiser {best_cost_model['Modèle']} (meilleur coût)")

print()
print("="*80)

# ================================================================================
# SAUVEGARDE DU TABLEAU


# Sauvegarde du DataFrame complet pour analyse ultérieure
df_comparison.to_csv("graphiques/comparison_baseline_models.csv", index=False)
print()
print("Tableau complet sauvegardé : graphiques/comparison_baseline_models.csv")

# ================================================================================
# VISUALISATION GRAPHIQUE


print()
print("GÉNÉRATION DES VISUALISATIONS COMPARATIVES...")
print()

# Récupérer données brutes pour les graphiques
df_plot = df_comparison.copy()

# Graphique Coût vs AUC
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#e74c3c' if i == 0 else '#3498db' for i in range(len(df_plot))]
sizes = [200 if i == 0 else 100 for i in range(len(df_plot))]

scatter = ax.scatter(
    df_plot['AUC-ROC'], 
    df_plot['Coût optimal'],
    c=colors,
    s=sizes,
    alpha=0.7,
    edgecolors='black',
    linewidth=1.5
)

# Annoter chaque point
for idx, row in df_plot.iterrows():
    ax.annotate(
        row['Modèle'].replace('_', '\n'),
        (row['AUC-ROC'], row['Coût optimal']),
        xytext=(10, 10),
        textcoords='offset points',
        fontsize=9,
        bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.3)
    )

ax.set_xlabel('AUC-ROC (Performance)', fontsize=12, fontweight='bold')
ax.set_ylabel('Coût Métier Optimal', fontsize=12, fontweight='bold')
ax.set_title('Comparaison des Modèles Baseline\nCoût Métier vs Performance (AUC)', 
             fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)

# Zones de préférence
ax.axhline(y=df_plot['Coût optimal'].min(), color='green', linestyle='--', 
          alpha=0.5, label=f'Meilleur coût: {df_plot["Coût optimal"].min():,.0f}')
ax.axvline(x=df_plot['AUC-ROC'].max(), color='blue', linestyle='--', 
          alpha=0.5, label=f'Meilleur AUC: {df_plot["AUC-ROC"].max():.4f}')

ax.legend(fontsize=10)
plt.tight_layout()
plt.savefig('graphiques/comparison_cost_vs_auc.png', dpi=100, bbox_inches='tight')
plt.close()

print("   Graphique 1 sauvegardé : graphiques/comparison_cost_vs_auc.png")

# Graphique en barres des coûts
fig, ax = plt.subplots(figsize=(12, 6))

x_pos = np.arange(len(df_plot))
bars = ax.bar(x_pos, df_plot['Coût optimal'], color=colors, alpha=0.7, edgecolor='black')

# Ajouter les valeurs sur les barres
for i, (bar, cost) in enumerate(zip(bars, df_plot['Coût optimal'])):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{cost:,.0f}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

ax.set_xlabel('Modèle', fontsize=12, fontweight='bold')
ax.set_ylabel('Coût Métier Optimal', fontsize=12, fontweight='bold')
ax.set_title('Comparaison des Coûts Métiers Optimaux\nObjectif : Minimiser le coût', 
             fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels([m.replace('_', '\n') for m in df_plot['Modèle']], fontsize=10)
ax.grid(axis='y', alpha=0.3)

# Ligne de référence (meilleur coût)
ax.axhline(y=df_plot['Coût optimal'].min(), color='green', linestyle='--', 
          linewidth=2, label=f'Meilleur coût: {df_plot["Coût optimal"].min():,.0f}')
ax.legend(fontsize=10)

plt.tight_layout()
plt.savefig('graphiques/comparison_costs_bar.png', dpi=100, bbox_inches='tight')
plt.close()

print("   Graphique 2 sauvegardé : graphiques/comparison_costs_bar.png")

# Graphique radar (métriques multiples)
from math import pi

categories = ['AUC-ROC', 'Recall', 'Precision', 'F1-Score']
N = len(categories)

# Normaliser les métriques sur [0, 1]
df_radar = df_plot.copy()
df_radar['AUC_norm'] = df_radar['AUC-ROC']
df_radar['Recall_norm'] = df_radar['Recall (optimal)']
df_radar['Precision_norm'] = df_radar['Precision (optimal)']
df_radar['F1_norm'] = df_radar['F1-Score (optimal)']

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

colors_radar = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12']

for idx, (_, row) in enumerate(df_radar.iterrows()):
    values = [
        row['AUC_norm'],
        row['Recall_norm'],
        row['Precision_norm'],
        row['F1_norm']
    ]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, 
            label=row['Modèle'], color=colors_radar[idx % len(colors_radar)])
    ax.fill(angles, values, alpha=0.15, color=colors_radar[idx % len(colors_radar)])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)
ax.set_ylim(0, 1)
ax.set_title('Comparaison Multi-Métriques des Modèles Baseline', 
             fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)
ax.grid(True)

plt.tight_layout()
plt.savefig('graphiques/comparison_radar.png', dpi=100, bbox_inches='tight')
plt.close()

print("   Graphique 3 sauvegardé : graphiques/comparison_radar.png")

print()
print("="*80)
print()
print("FICHIERS GÉNÉRÉS:")
print("   - graphiques/comparison_baseline_models.csv (données brutes)")
print("   - graphiques/comparison_cost_vs_auc.png (scatter plot)")
print("   - graphiques/comparison_costs_bar.png (barres)")
print("   - graphiques/comparison_radar.png (multi-métriques)")
print()

COMPARAISON DES MODÈLES BASELINE
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.

Nombre de runs baseline trouvés : 4

TABLEAU COMPARATIF COMPLET

1. COMPARAISON SUR LE COÛT MÉTIER (CRITÈRE PRINCIPAL)
--------------------------------------------------------------------------------
                   Modèle Coût (seuil 0.5) Seuil optimal Coût optimal Économie Économie %
         XGBoost_Balanced           30,506          0.48       30,249      257       0.8%
        LightGBM_Balanced           30,988          0.52       30,876      112       0.4%
LogisticRegression_Scaled           31,436          0.53       31,408       28       0.1%
    RandomForest_Balanced           32,754          0.48       32,671       83       0.3%

2. MÉTRIQUES TECHNIQUES (AUC, ACCURACY)
--------------------------------------------------------------------------------
                   Modèle AUC-ROC CV AUC

<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Optimisation LightGBM et XGBOOST</h2>

<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Optimisation LightGBM</h3>

In [23]:
# OPTIMISATION LIGHTGBM AVEC OPTUNA


print("="*80)
print("OPTIMISATION HYPERPARAMETRES - LIGHTGBM")
print("="*80)
print()

# Configuration MLflow
mlflow.set_experiment("home_credit_scoring")

# Fonction objective pour Optuna (LightGBM)
def objective_lightgbm(trial):
    """
    Fonction objective pour optimiser LightGBM avec Optuna.
    Retourne le cout metier optimal a minimiser.
    """
    
    # Définition de l'espace de recherche
    params = {
        # Nombre d'arbres (impact: fort)
        'n_estimators': trial.suggest_int('n_estimators', 100, 300, step=50),
        
        # Profondeur maximale (impact: fort)
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        
        # Taux d'apprentissage (impact: critique)
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        
        # Nombre de feuilles (impact: moyen)
        'num_leaves': trial.suggest_int('num_leaves', 20, 80, step=10),
        
        # Échantillons minimum par feuille (impact: moyen)
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=10),
        
        # Sous-échantillonnage des lignes (impact: faible-moyen)
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1),
        
        # Sous-échantillonnage des colonnes (impact: faible-moyen)
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.1),
        
        # Régularisation L1 (impact: faible)
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0, step=0.1),
        
        # Régularisation L2 (impact: faible)
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0, step=0.1),
        
        # Paramètres fixes
        'class_weight': 'balanced',
        'random_state': 42,
        'verbose': -1,
        'n_jobs': -1
    }
    
    # Créer le modèle
    model = LGBMClassifier(**params)
    
    # Entraîner avec fonction
    results = train_and_log_model(
        model=model,
        model_name=f"LightGBM_Optuna_Trial_{trial.number}",
        X_train=X_train_lgbm,
        y_train=y_train,
        X_val=X_val_lgbm,
        y_val=y_val,
        cv_folds=5,
        tags={
            "type": "hyperparameters_optuna",
            "model_family": "LightGBM",
            "trial_number": trial.number,
            "optimization_method": "optuna"
        },
        description=f"Optuna optimization - Trial {trial.number}"
    )
    
    # Retourner le coût métier optimal à minimiser
    return results['cost_at_optimal_threshold']

print("Fonction objective LightGBM créée")
print()

OPTIMISATION HYPERPARAMETRES - LIGHTGBM

Fonction objective LightGBM créée



In [24]:
# LANCEMENT OPTIMISATION LIGHTGBM

print("LANCEMENT DE L'OPTIMISATION LIGHTGBM")
print("-"*80)
print()

# Créer l'étude Optuna pour LightGBM
study_lgbm = optuna.create_study(
    direction='minimize',                        # Minimiser le coût métier
    study_name='LightGBM_Cost_Optimization',
    sampler=optuna.samplers.TPESampler(seed=42)  # Bayesian optimization
)

print(f"Étude créée : {study_lgbm.study_name}")
print(f"Direction : {study_lgbm.direction.name}")
print(f"Sampler : {type(study_lgbm.sampler).__name__}")
print()

# Lancer l'optimisation
print(f"Démarrage de l'optimisation (50 trials)...")
print(f"Durée estimée : 30-40 minutes")
print(f"Chaque trial prend environ 40-50 secondes")
print()

import time
start_time_lgbm = time.time()

# Optimisation avec barre de progression
study_lgbm.optimize(
    objective_lightgbm,
    n_trials=50,
    n_jobs=1,  # Séquentiel (obligatoire avec MLflow)
    show_progress_bar=True
)

end_time_lgbm = time.time()
duration_lgbm = end_time_lgbm - start_time_lgbm

print()
print("="*80)
print("OPTIMISATION LIGHTGBM TERMINÉE")
print("="*80)
print()
print(f"Durée totale : {duration_lgbm/60:.1f} minutes ({duration_lgbm:.0f} secondes)")
print(f"Nombre de trials : {len(study_lgbm.trials)}")
print()

# Résultats de l'optimisation
print("MEILLEURS RÉSULTATS TROUVÉS:")
print("-"*80)
print(f"Meilleur coût métier : {study_lgbm.best_value:,.0f}")
print()
print("Meilleurs hyperparamètres :")
for param_name, param_value in study_lgbm.best_params.items():
    print(f"   - {param_name:<20} : {param_value}")
print()

# Comparer avec baseline
baseline_cost_lgbm = 30876  # Coût baseline LightGBM
improvement_lgbm = baseline_cost_lgbm - study_lgbm.best_value
improvement_pct_lgbm = (improvement_lgbm / baseline_cost_lgbm) * 100

print("COMPARAISON AVEC BASELINE:")
print(f"   - Baseline (défaut)      : {baseline_cost_lgbm:,}")
print(f"   - Optimisé (Optuna)      : {study_lgbm.best_value:,.0f}")
print(f"   - Amélioration           : {improvement_lgbm:,.0f} (-{improvement_pct_lgbm:.2f}%)")
print()

# Sauvegarder l'étude
import pickle
with open('graphiques/study_lightgbm.pkl', 'wb') as f:
    pickle.dump(study_lgbm, f)
print("Étude sauvegardée : graphiques/study_lightgbm.pkl")
print()

[I 2025-10-09 14:28:33,847] A new study created in memory with name: LightGBM_Cost_Optimization


LANCEMENT DE L'OPTIMISATION LIGHTGBM
--------------------------------------------------------------------------------

Étude créée : LightGBM_Cost_Optimization
Direction : MINIMIZE
Sampler : TPESampler

Démarrage de l'optimisation (50 trials)...
Durée estimée : 30-40 minutes
Chaque trial prend environ 40-50 secondes



  0%|                                                                                               | 0/50 [00:00<?, ?it/s]


MODELE : LightGBM_Optuna_Trial_0
Run ID : b47681ad3e534257830cd66f985dd6cd

CROSS-VALIDATION STRATIFIEE (5 folds)
────────────────────────────────────────────────────────────────────────────────
Resultats Cross-Validation (5 folds) :
   - AUC-ROC   : 0.7785 (+/-0.0018)
   - Accuracy  : 0.7413 (+/-0.0018)
   - Precision : 0.1879 (+/-0.0020)
   - Recall    : 0.6636 (+/-0.0050)
   - F1-Score  : 0.2928 (+/-0.0030)

ENTRAINEMENT FINAL
────────────────────────────────────────────────────────────────────────────────
Entrainement sur 246,008 echantillons...
Termine en 52.59s

EVALUATION SUR VALIDATION SET
────────────────────────────────────────────────────────────────────────────────

Metriques Validation (Classe 1: Defaut) :
   - AUC-ROC   : 0.7824
   - Accuracy  : 0.7368
   - Precision : 0.1874
   - Recall    : 0.6773
   - F1-Score  : 0.2935

   Metriques Classe 0 (Pas de Defaut) :
   - Precision : 0.9632
   - Recall    : 0.7420
   - F1-Score  : 0.8383

Matrice de Confusion :
   - TN (Vrai

2025/10/09 14:29:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 14:29:28 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


...

   Modele enregistre dans MLflow

RESULTATS FINAUX - LightGBM_Optuna_Trial_48
Temps d'entrainement : 65.41s
Temps optimisation seuil : 0.81s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7786 (+/-0.0020)
   - Accuracy  : 0.7378 (+/-0.0011)
   - Precision : 0.1870 (+/-0.0015)
   - Recall    : 0.6717 (+/-0.0056)
   - F1-Score  : 0.2926 (+/-0.0023)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7820
   - Accuracy  : 0.7342
   - Precision : 0.1858
   - Recall    : 0.6777
   - F1-Score  : 0.2916
   - TN/FP/FN/TP : 41,792 / 14,746 / 1,600 / 3,365
   - Cout metier : 30,746

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.52
   - Cout au seuil optimal  : 30,732 (vs 30,746 au seuil 0.5)
   - Economie realisee      : 14 (-0.0%)
   - Recall au seuil optimal: 0.6520
   - Defauts detectes       : 3,237 (vs 3,365)
   - Defauts manques        : 1,728 (vs 1,600)
Run termine - ID: 7b461fb1c4dd40da84b7f9a98aae8d18

[I 2025-10-09 15:11:58,309] Trial 48 finished with value: 30732.0 

2025/10/09 15:13:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 15:13:07 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


   Scores CV sauvegardes

SAUVEGARDE DU MODELE
────────────────────────────────────────────────────────────────────────────────


Best trial: 41. Best value: 30118: 100%|███████████████████████████████████████████████████| 50/50 [44:33<00:00, 53.47s/it]

   Modele enregistre dans MLflow

RESULTATS FINAUX - LightGBM_Optuna_Trial_49
Temps d'entrainement : 67.10s
Temps optimisation seuil : 0.81s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7804 (+/-0.0022)
   - Accuracy  : 0.7489 (+/-0.0021)
   - Precision : 0.1919 (+/-0.0022)
   - Recall    : 0.6572 (+/-0.0046)
   - F1-Score  : 0.2971 (+/-0.0030)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7828
   - Accuracy  : 0.7448
   - Precision : 0.1908
   - Recall    : 0.6667
   - F1-Score  : 0.2967
   - TN/FP/FN/TP : 42,498 / 14,040 / 1,655 / 3,310
   - Cout metier : 30,590

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.51
   - Cout au seuil optimal  : 30,492 (vs 30,590 au seuil 0.5)
   - Economie realisee      : 98 (-0.3%)
   - Recall au seuil optimal: 0.6562
   - Defauts detectes       : 3,258 (vs 3,310)
   - Defauts manques        : 1,707 (vs 1,655)
Run termine - ID: bf49e8bd624b431c80255a4ae34e3dd2

[I 2025-10-09 15:13:07,589] Trial 49 finished with value: 30492.0 

In [37]:
# VISUALISATIONS MATPLOTLIB - LIGHTGBM

print("GÉNÉRATION DES VISUALISATIONS LIGHTGBM")
print("-"*80)
print()

# Historique d'optimisation
fig, ax = plt.subplots(figsize=(12, 6))

trial_numbers = [trial.number for trial in study_lgbm.trials]
trial_values = [trial.value for trial in study_lgbm.trials]
best_values = [study_lgbm.best_trials[0].value if i == 0 else min(trial_values[:i+1]) 
               for i in range(len(trial_values))]

ax.plot(trial_numbers, trial_values, 'o-', alpha=0.6, label='Coût par trial', color='#3498db')
ax.plot(trial_numbers, best_values, '-', linewidth=2, label='Meilleur coût', color='#2ecc71')
ax.axhline(y=study_lgbm.best_value, color='red', linestyle='--', 
          label=f'Optimal: {study_lgbm.best_value:,.0f}', alpha=0.7)

ax.set_xlabel('Trial', fontsize=12, fontweight='bold')
ax.set_ylabel('Coût métier optimal', fontsize=12, fontweight='bold')
ax.set_title('LightGBM - Historique d\'optimisation Optuna', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("graphiques/optuna_lgbm_history.png", dpi=100, bbox_inches='tight')
plt.close()
print("   Graphique 1 : graphiques/optuna_lgbm_history.png")

# Importance des hyperparamètres (calcul simplifié)
param_names = list(study_lgbm.best_params.keys())
param_importance = {}

for param in param_names:
    param_values = []
    costs = []
    for trial in study_lgbm.trials:
        if trial.state == optuna.trial.TrialState.COMPLETE:
            param_values.append(trial.params.get(param))
            costs.append(trial.value)
    
    # Calculer corrélation simple (approximation de l'importance)
    if len(set(param_values)) > 1:  # Si le paramètre varie
        correlation = abs(np.corrcoef(param_values, costs)[0, 1])
        param_importance[param] = correlation if not np.isnan(correlation) else 0
    else:
        param_importance[param] = 0

# Trier par importance
sorted_params = sorted(param_importance.items(), key=lambda x: x[1], reverse=True)

fig, ax = plt.subplots(figsize=(10, 6))
params = [p[0] for p in sorted_params]
importances = [p[1] for p in sorted_params]

ax.barh(params, importances, color='#3498db', alpha=0.7)
ax.set_xlabel('Importance (corrélation absolue)', fontsize=12, fontweight='bold')
ax.set_title('LightGBM - Importance des hyperparamètres', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig("graphiques/optuna_lgbm_importance.png", dpi=100, bbox_inches='tight')
plt.close()
print("   Graphique 2 : graphiques/optuna_lgbm_importance.png")

# Distribution des 3 paramètres les plus importants
top_3_params = [p[0] for p in sorted_params[:3]]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, param in enumerate(top_3_params):
    param_vals = [trial.params.get(param) for trial in study_lgbm.trials 
                  if trial.state == optuna.trial.TrialState.COMPLETE]
    costs = [trial.value for trial in study_lgbm.trials 
             if trial.state == optuna.trial.TrialState.COMPLETE]
    
    scatter = axes[idx].scatter(param_vals, costs, alpha=0.6, c=costs, 
                               cmap='RdYlGn_r', s=50, edgecolors='black', linewidth=0.5)
    axes[idx].axhline(y=study_lgbm.best_value, color='red', linestyle='--', 
                     alpha=0.7, label=f'Optimal: {study_lgbm.best_value:,.0f}')
    axes[idx].set_xlabel(param, fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Coût métier' if idx == 0 else '', fontsize=11)
    axes[idx].set_title(f'Impact de {param}', fontsize=12, fontweight='bold')
    axes[idx].grid(alpha=0.3)
    axes[idx].legend(fontsize=9)
    
    plt.colorbar(scatter, ax=axes[idx], label='Coût')

plt.suptitle('LightGBM - Impact des principaux hyperparamètres', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("graphiques/optuna_lgbm_params_impact.png", dpi=100, bbox_inches='tight')
plt.close()
print("   Graphique 3 : graphiques/optuna_lgbm_params_impact.png")

print()
print("Visualisations LightGBM terminées")
print()

GÉNÉRATION DES VISUALISATIONS LIGHTGBM
--------------------------------------------------------------------------------

   Graphique 1 : graphiques/optuna_lgbm_history.png
   Graphique 2 : graphiques/optuna_lgbm_importance.png
   Graphique 3 : graphiques/optuna_lgbm_params_impact.png

Visualisations LightGBM terminées



In [23]:
# PRÉPARATION DONNÉES POUR LIGHTGBM

# Nettoyer les noms de colonnes pour LightGBM (enlever caractères spéciaux)
X_train_lgbm = X_train_encoded.copy()
X_val_lgbm = X_val_encoded.copy()

# Remplacer les caractères spéciaux dans les noms de colonnes
X_train_lgbm.columns = X_train_lgbm.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)
X_val_lgbm.columns = X_val_lgbm.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)

print(f"Noms de colonnes nettoyés pour LightGBM")
print(f"Exemple avant: {X_train_encoded.columns[0]}")
print(f"Exemple après: {X_train_lgbm.columns[0]}")

# ========================================
# ENTRAÎNEMENT LIGHTGBM
# ========================================


# Configuration
mlflow.set_experiment("home_credit_scoring")

# Modèle
model = LGBMClassifier(
    n_estimators=300,
    class_weight='balanced',
    learning_rate=0.058264831213179456,
    max_depth=7,
    num_leaves=40,
    min_child_samples=30,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=0.9,
    reg_lambda=0.8,
    random_state=42
    
)

# Lancer l'entraînement
results = train_and_log_model(
    model=model,
    model_name="LightGBM_Balanced",
    X_train=X_train_lgbm,
    y_train=y_train,
    X_val=X_val_lgbm,
    y_val=y_val,
    cv_folds=5,
    description="LightGBM avec class_weight balanced",
    tags={"preprocessing": "StandardScaler", "type": "hyperparameters_optimised"}
)

# Résultats
print(f"AUC CV: {results['cv_auc_roc_mean']:.4f}")
print(f"AUC Val: {results['val_auc_roc']:.4f}")

Noms de colonnes nettoyés pour LightGBM
Exemple avant: SK_ID_CURR
Exemple après: SK_ID_CURR

MODELE : LightGBM_Balanced
Run ID : 581b5c77ed604287975ca6f77e414567

CROSS-VALIDATION STRATIFIEE (5 folds)
────────────────────────────────────────────────────────────────────────────────
Resultats Cross-Validation (5 folds) :
   - AUC-ROC   : 0.7804 (+/-0.0017)
   - Accuracy  : 0.7543 (+/-0.0013)
   - Precision : 0.1947 (+/-0.0013)
   - Recall    : 0.6518 (+/-0.0044)
   - F1-Score  : 0.2999 (+/-0.0019)

ENTRAINEMENT FINAL
────────────────────────────────────────────────────────────────────────────────
Entrainement sur 246,008 echantillons...
[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.178108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 105842
[LightGBM] [Info] Number of data points in the train set: 246008, number of used featur

2025/10/20 13:08:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/20 13:08:03 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


   Scores CV sauvegardes

SAUVEGARDE DU MODELE
────────────────────────────────────────────────────────────────────────────────
   Modele enregistre dans MLflow

RESULTATS FINAUX - LightGBM_Balanced
Temps d'entrainement : 59.13s
Temps optimisation seuil : 0.86s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7804 (+/-0.0017)
   - Accuracy  : 0.7543 (+/-0.0013)
   - Precision : 0.1947 (+/-0.0013)
   - Recall    : 0.6518 (+/-0.0044)
   - F1-Score  : 0.2999 (+/-0.0019)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7828
   - Accuracy  : 0.7510
   - Precision : 0.1954
   - Recall    : 0.6687
   - F1-Score  : 0.3025
   - TN/FP/FN/TP : 42,870 / 13,668 / 1,645 / 3,320
   - Cout metier : 30,118

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.50
   - Cout au seuil optimal  : 30,118 (vs 30,118 au seuil 0.5)
   - Economie realisee      : 0 (-0.0%)
   - Recall au seuil optimal: 0.6687
   - Defauts detectes       : 3,320 (vs 3,320)
   - Defauts manques        : 1,645 (vs 1,645

<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Optimisation XGBOOST</h3>

In [40]:
# PRÉPARATION DONNÉES POUR XGBOOST


print("NETTOYAGE DES NOMS DE COLONNES POUR XGBOOST")
print("="*80)

# Créer des copies pour XGBoost
X_train_xgb = X_train_encoded.copy()
X_val_xgb = X_val_encoded.copy()

# Nettoyer les noms de colonnes (enlever caractères spéciaux)
X_train_xgb.columns = X_train_xgb.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)
X_val_xgb.columns = X_val_xgb.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)

print(f"Noms de colonnes nettoyés pour XGBoost")
print(f"Exemple avant : BUREAU_CREDIT_ACTIVE_<LAMBDA_0>")
print(f"Exemple après : BUREAU_CREDIT_ACTIVE__LAMBDA_0_")
print()
print(f"Dataset XGBoost prêt : {X_train_xgb.shape}")
print()

NETTOYAGE DES NOMS DE COLONNES POUR XGBOOST
✅ Noms de colonnes nettoyés pour XGBoost
Exemple avant : BUREAU_CREDIT_ACTIVE_<LAMBDA_0>
Exemple après : BUREAU_CREDIT_ACTIVE__LAMBDA_0_

Dataset XGBoost prêt : (246008, 912)



In [41]:
# OPTIMISATION XGBOOST AVEC OPTUNA


print("="*80)
print("PHASE 2.2 : OPTIMISATION HYPERPARAMÈTRES - XGBOOST")
print("="*80)
print()

# Fonction objective pour Optuna (XGBoost)
def objective_xgboost(trial):
    """
    Fonction objective pour optimiser XGBoost avec Optuna.
    Basée sur vos hyperparamètres baseline :
    - n_estimators: 150
    - max_depth: 7
    - learning_rate: 0.05
    - scale_pos_weight: 11.4
    
    Retourne le cout metier optimal a minimiser.
    """
    
    # Calculer scale_pos_weight (ratio classes)
    n_positive = (y_train == 1).sum()
    n_negative = (y_train == 0).sum()
    scale_pos_weight_value = n_negative / n_positive  # ~11.4
    
    # Définition de l'espace de recherche (autour de vos valeurs baseline)
    params = {
        # Nombre d'arbres (baseline: 150)
        'n_estimators': trial.suggest_int('n_estimators', 100, 300, step=50),
        
        # Profondeur maximale (baseline: 7)
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        
        # Taux d'apprentissage (baseline: 0.05)
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        
        # Poids minimum par enfant (impact sur déséquilibre)
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        
        # Gamma (régularisation)
        'gamma': trial.suggest_float('gamma', 0.0, 0.5, step=0.1),
        
        # Sous-échantillonnage des lignes
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1),
        
        # Sous-échantillonnage des colonnes par arbre
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.1),
        
        # Sous-échantillonnage des colonnes par niveau
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0, step=0.1),
        
        # Régularisation L1
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0, step=0.1),
        
        # Régularisation L2
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0, step=0.1),
        
        # Paramètres fixes (comme votre baseline)
        'scale_pos_weight': scale_pos_weight_value,  # Gestion déséquilibre (11.4)
        'random_state': 42,
        'tree_method': 'hist',        # Plus rapide pour grands datasets
        'eval_metric': 'logloss',     # Métrique standard classification
        'verbosity': 0,               # Désactiver logs verbeux
        'n_jobs': -1                  # Parallélisation
    }
    
    # Créer le modèle
    model = XGBClassifier(**params)
    
    # Entraîner avec fonction
    results = train_and_log_model(
        model=model,
        model_name=f"XGBoost_Optuna_Trial_{trial.number}",
        X_train=X_train_xgb,
        y_train=y_train,
        X_val=X_val_xgb,
        y_val=y_val,
        cv_folds=5,
        tags={
            "type": "hyperparameters_optuna",
            "model_family": "XGBoost",
            "trial_number": trial.number,
            "optimization_method": "optuna",
            "preprocessing": "column_names_cleaned"
        },
        description=f"Optuna optimization - Trial {trial.number} - Baseline: n_est=150, max_depth=7, lr=0.05"
    )
    
    # Retourner le coût métier optimal à minimiser
    return results['cost_at_optimal_threshold']

print("Fonction objective XGBoost créée")
print()
print("ESPACE DE RECHERCHE DÉFINI:")
print("  • n_estimators     : [100, 150, 200, 250, 300] (baseline: 150)")
print("  • max_depth        : [4, 5, 6, 7, 8, 9, 10] (baseline: 7)")
print("  • learning_rate    : [0.01 ... 0.1] log-scale (baseline: 0.05)")
print("  • min_child_weight : [1 ... 7]")
print("  • gamma            : [0.0 ... 0.5]")
print("  • subsample        : [0.6 ... 1.0]")
print("  • colsample_bytree : [0.6 ... 1.0]")
print("  • colsample_bylevel: [0.6 ... 1.0]")
print("  • reg_alpha        : [0.0 ... 1.0]")
print("  • reg_lambda       : [0.0 ... 1.0]")
print()
print("PARAMÈTRES FIXES:")
print("  • scale_pos_weight : 11.4 (ratio classes)")
print("  • tree_method      : 'hist' (rapide)")
print("  • eval_metric      : 'logloss'")
print()

PHASE 2.2 : OPTIMISATION HYPERPARAMÈTRES - XGBOOST

✅ Fonction objective XGBoost créée

ESPACE DE RECHERCHE DÉFINI:
  • n_estimators     : [100, 150, 200, 250, 300] (baseline: 150)
  • max_depth        : [4, 5, 6, 7, 8, 9, 10] (baseline: 7)
  • learning_rate    : [0.01 ... 0.1] log-scale (baseline: 0.05)
  • min_child_weight : [1 ... 7]
  • gamma            : [0.0 ... 0.5]
  • subsample        : [0.6 ... 1.0]
  • colsample_bytree : [0.6 ... 1.0]
  • colsample_bylevel: [0.6 ... 1.0]
  • reg_alpha        : [0.0 ... 1.0]
  • reg_lambda       : [0.0 ... 1.0]

PARAMÈTRES FIXES:
  • scale_pos_weight : 11.4 (ratio classes)
  • tree_method      : 'hist' (rapide)
  • eval_metric      : 'logloss'



In [42]:
# LANCEMENT OPTIMISATION XGBOOST

print("LANCEMENT DE L'OPTIMISATION XGBOOST")
print("-"*80)
print()

# Créer l'étude Optuna pour XGBoost
study_xgb = optuna.create_study(
    direction='minimize',  # Minimiser le coût métier
    study_name='XGBoost_Cost_Optimization',
    sampler=optuna.samplers.TPESampler(seed=42)  # Bayesian optimization
)

print(f"Étude créée : {study_xgb.study_name}")
print(f"Direction : {study_xgb.direction.name}")
print(f"Sampler : {type(study_xgb.sampler).__name__}")
print()

# Lancer l'optimisation
print(f"Démarrage de l'optimisation (50 trials)...")
print(f"Durée estimée : 40-50 minutes")
print(f"Chaque trial prend environ 50-60 secondes")
print()

start_time_xgb = time.time()

# Optimisation avec barre de progression
study_xgb.optimize(
    objective_xgboost,
    n_trials=50,
    n_jobs=1,  # Séquentiel (obligatoire avec MLflow)
    show_progress_bar=True
)

end_time_xgb = time.time()
duration_xgb = end_time_xgb - start_time_xgb

print()
print("="*80)
print("OPTIMISATION XGBOOST TERMINÉE")
print("="*80)
print()
print(f"Durée totale : {duration_xgb/60:.1f} minutes ({duration_xgb:.0f} secondes)")
print(f"Nombre de trials : {len(study_xgb.trials)}")
print()

# Résultats de l'optimisation
print("MEILLEURS RÉSULTATS TROUVÉS:")
print("-"*80)
print(f"Meilleur coût métier : {study_xgb.best_value:,.0f}")
print()
print("Meilleurs hyperparamètres :")
for param_name, param_value in study_xgb.best_params.items():
    print(f"   - {param_name:<20} : {param_value}")
print()

# Comparer avec baseline
baseline_cost_xgb = 30742  # Coût baseline XGBoost
improvement_xgb = baseline_cost_xgb - study_xgb.best_value
improvement_pct_xgb = (improvement_xgb / baseline_cost_xgb) * 100

print("COMPARAISON AVEC BASELINE:")
print(f"   - Baseline (défaut)      : {baseline_cost_xgb:,}")
print(f"   - Optimisé (Optuna)      : {study_xgb.best_value:,.0f}")
print(f"   - Amélioration           : {improvement_xgb:,.0f} (-{improvement_pct_xgb:.2f}%)")
print()

# Sauvegarder l'étude
with open('graphiques/study_xgboost.pkl', 'wb') as f:
    pickle.dump(study_xgb, f)
print("Étude sauvegardée : graphiques/study_xgboost.pkl")
print()

[I 2025-10-09 15:59:35,272] A new study created in memory with name: XGBoost_Cost_Optimization


LANCEMENT DE L'OPTIMISATION XGBOOST
--------------------------------------------------------------------------------

Étude créée : XGBoost_Cost_Optimization
Direction : MINIMIZE
Sampler : TPESampler

Démarrage de l'optimisation (50 trials)...
Durée estimée : 40-50 minutes
Chaque trial prend environ 50-60 secondes



  0%|                                                                                               | 0/50 [00:00<?, ?it/s]


MODELE : XGBoost_Optuna_Trial_0
Run ID : ab3c962e07e045b58613327186476eb1

CROSS-VALIDATION STRATIFIEE (5 folds)
────────────────────────────────────────────────────────────────────────────────
Resultats Cross-Validation (5 folds) :
   - AUC-ROC   : 0.7622 (+/-0.0033)
   - Accuracy  : 0.8257 (+/-0.0022)
   - Precision : 0.2264 (+/-0.0037)
   - Recall    : 0.4791 (+/-0.0063)
   - F1-Score  : 0.3074 (+/-0.0044)

ENTRAINEMENT FINAL
────────────────────────────────────────────────────────────────────────────────
Entrainement sur 246,008 echantillons...
Termine en 78.59s

EVALUATION SUR VALIDATION SET
────────────────────────────────────────────────────────────────────────────────

Metriques Validation (Classe 1: Defaut) :
   - AUC-ROC   : 0.7669
   - Accuracy  : 0.8158
   - Precision : 0.2218
   - Recall    : 0.5110
   - F1-Score  : 0.3093

   Metriques Classe 0 (Pas de Defaut) :
   - Precision : 0.9515
   - Recall    : 0.8425
   - F1-Score  : 0.8937

Matrice de Confusion :
   - TN (Vrai 

2025/10/09 16:00:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 16:00:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


...

   Modele enregistre dans MLflow

RESULTATS FINAUX - XGBoost_Optuna_Trial_48
Temps d'entrainement : 82.03s
Temps optimisation seuil : 0.82s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7780 (+/-0.0020)
   - Accuracy  : 0.7617 (+/-0.0020)
   - Precision : 0.1971 (+/-0.0024)
   - Recall    : 0.6347 (+/-0.0059)
   - F1-Score  : 0.3008 (+/-0.0034)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7822
   - Accuracy  : 0.7553
   - Precision : 0.1951
   - Recall    : 0.6495
   - F1-Score  : 0.3000
   - TN/FP/FN/TP : 43,231 / 13,307 / 1,740 / 3,225
   - Cout metier : 30,707

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.49
   - Cout au seuil optimal  : 30,616 (vs 30,707 au seuil 0.5)
   - Economie realisee      : 91 (-0.3%)
   - Recall au seuil optimal: 0.6630
   - Defauts detectes       : 3,292 (vs 3,225)
   - Defauts manques        : 1,673 (vs 1,740)
Run termine - ID: a04e90ccba954b3abf2f0e165d0b46f0

[I 2025-10-09 17:00:33,203] Trial 48 finished with value: 30616.0 a

2025/10/09 17:01:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 17:01:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


   Scores CV sauvegardes

SAUVEGARDE DU MODELE
────────────────────────────────────────────────────────────────────────────────


Best trial: 43. Best value: 30279: 100%|█████████████████████████████████████████████████| 50/50 [1:01:48<00:00, 74.17s/it]

   Modele enregistre dans MLflow

RESULTATS FINAUX - XGBoost_Optuna_Trial_49
Temps d'entrainement : 48.13s
Temps optimisation seuil : 0.81s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7707 (+/-0.0015)
   - Accuracy  : 0.7799 (+/-0.0030)
   - Precision : 0.2026 (+/-0.0031)
   - Recall    : 0.5880 (+/-0.0035)
   - F1-Score  : 0.3014 (+/-0.0038)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7761
   - Accuracy  : 0.7731
   - Precision : 0.2017
   - Recall    : 0.6125
   - F1-Score  : 0.3035
   - TN/FP/FN/TP : 44,505 / 12,033 / 1,924 / 3,041
   - Cout metier : 31,273

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.45
   - Cout au seuil optimal  : 31,131 (vs 31,273 au seuil 0.5)
   - Economie realisee      : 142 (-0.5%)
   - Recall au seuil optimal: 0.6755
   - Defauts detectes       : 3,354 (vs 3,041)
   - Defauts manques        : 1,611 (vs 1,924)
Run termine - ID: 312b7f028da34c3cb113efb119d24eaf

[I 2025-10-09 17:01:23,761] Trial 49 finished with value: 31131.0 

In [43]:
# VISUALISATIONS MATPLOTLIB - XGBOOST

print("GÉNÉRATION DES VISUALISATIONS XGBOOST")
print("-"*80)
print()

# Historique d'optimisation
fig, ax = plt.subplots(figsize=(12, 6))

trial_numbers = [trial.number for trial in study_xgb.trials]
trial_values = [trial.value for trial in study_xgb.trials]
best_values = [study_xgb.best_trials[0].value if i == 0 else min(trial_values[:i+1]) 
               for i in range(len(trial_values))]

ax.plot(trial_numbers, trial_values, 'o-', alpha=0.6, label='Coût par trial', color='#3498db')
ax.plot(trial_numbers, best_values, '-', linewidth=2, label='Meilleur coût', color='#2ecc71')
ax.axhline(y=study_xgb.best_value, color='red', linestyle='--', 
          label=f'Optimal: {study_xgb.best_value:,.0f}', alpha=0.7)

ax.set_xlabel('Trial', fontsize=12, fontweight='bold')
ax.set_ylabel('Coût métier optimal', fontsize=12, fontweight='bold')
ax.set_title('XGBoost - Historique d\'optimisation Optuna', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("graphiques/optuna_xgb_history.png", dpi=100, bbox_inches='tight')
plt.close()
print("   Graphique 1 : graphiques/optuna_xgb_history.png")

# Importance des hyperparamètres
param_names = list(study_xgb.best_params.keys())
param_importance = {}

for param in param_names:
    param_values = []
    costs = []
    for trial in study_xgb.trials:
        if trial.state == optuna.trial.TrialState.COMPLETE:
            param_values.append(trial.params.get(param))
            costs.append(trial.value)
    
    # Calculer corrélation simple (approximation de l'importance)
    if len(set(param_values)) > 1:  # Si le paramètre varie
        correlation = abs(np.corrcoef(param_values, costs)[0, 1])
        param_importance[param] = correlation if not np.isnan(correlation) else 0
    else:
        param_importance[param] = 0

# Trier par importance
sorted_params = sorted(param_importance.items(), key=lambda x: x[1], reverse=True)

fig, ax = plt.subplots(figsize=(10, 6))
params = [p[0] for p in sorted_params]
importances = [p[1] for p in sorted_params]

ax.barh(params, importances, color='#e74c3c', alpha=0.7)
ax.set_xlabel('Importance (corrélation absolue)', fontsize=12, fontweight='bold')
ax.set_title('XGBoost - Importance des hyperparamètres', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig("graphiques/optuna_xgb_importance.png", dpi=100, bbox_inches='tight')
plt.close()
print("   Graphique 2 : graphiques/optuna_xgb_importance.png")

# 3. Distribution des 3 paramètres les plus importants
top_3_params = [p[0] for p in sorted_params[:3]]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, param in enumerate(top_3_params):
    param_vals = [trial.params.get(param) for trial in study_xgb.trials 
                  if trial.state == optuna.trial.TrialState.COMPLETE]
    costs = [trial.value for trial in study_xgb.trials 
             if trial.state == optuna.trial.TrialState.COMPLETE]
    
    scatter = axes[idx].scatter(param_vals, costs, alpha=0.6, c=costs, 
                               cmap='RdYlGn_r', s=50, edgecolors='black', linewidth=0.5)
    axes[idx].axhline(y=study_xgb.best_value, color='red', linestyle='--', 
                     alpha=0.7, label=f'Optimal: {study_xgb.best_value:,.0f}')
    axes[idx].set_xlabel(param, fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Coût métier' if idx == 0 else '', fontsize=11)
    axes[idx].set_title(f'Impact de {param}', fontsize=12, fontweight='bold')
    axes[idx].grid(alpha=0.3)
    axes[idx].legend(fontsize=9)
    
    plt.colorbar(scatter, ax=axes[idx], label='Coût')

plt.suptitle('XGBoost - Impact des principaux hyperparamètres', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("graphiques/optuna_xgb_params_impact.png", dpi=100, bbox_inches='tight')
plt.close()
print("   Graphique 3 : graphiques/optuna_xgb_params_impact.png")

print()
print("Visualisations XGBoost terminées")
print()

GÉNÉRATION DES VISUALISATIONS XGBOOST
--------------------------------------------------------------------------------

   Graphique 1 : graphiques/optuna_xgb_history.png
   Graphique 2 : graphiques/optuna_xgb_importance.png
   Graphique 3 : graphiques/optuna_xgb_params_impact.png

Visualisations XGBoost terminées



In [49]:
# PRÉPARATION DONNÉES POUR XGBOOST

print("NETTOYAGE DES NOMS DE COLONNES POUR XGBOOST")
print("="*70)

# Créer des copies pour XGBoost
X_train_xgb = X_train_encoded.copy()
X_val_xgb = X_val_encoded.copy()

# Nettoyer les noms de colonnes (enlever caractères spéciaux)
X_train_xgb.columns = X_train_xgb.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)
X_val_xgb.columns = X_val_xgb.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)

print(f"Noms de colonnes nettoyés pour XGBoost")
print(f"Exemple avant : BUREAU_CREDIT_ACTIVE_<LAMBDA_0>")
print(f"Exemple après : BUREAU_CREDIT_ACTIVE__LAMBDA_0_")
print()

# ========================================
# ENTRAÎNEMENT XGBOOST
# ========================================

# Configuration
mlflow.set_experiment("home_credit_scoring")

# Modèle XGBoost
model = XGBClassifier(
    n_estimators=300,
    scale_pos_weight=11.4,      # Ratio classes (91.9/8.1)
    learning_rate=0.06797992020440427,
    max_depth=6,
    min_child_weight=7,
    gamma=0.5,
    subsample=1.0,
    colsample_bytree=0.8,
    colsample_byleve=0.6,
    reg_alpha=0.8,
    reg_lambda=0.1,
    random_state=42
)

# Lancer l'entraînement
results = train_and_log_model(
    model=model,
    model_name="XGBoost_Balanced",
    X_train=X_train_xgb,        # Utilisation des données nettoyées
    y_train=y_train,
    X_val=X_val_xgb,            # Utilisation des données nettoyées
    y_val=y_val,
    cv_folds=5,
    description="XGBoost avec scale_pos_weight=11.4 (ratio classes)",
    tags={"preprocessing": "column_names_cleaned", "type": "hyperparameters_optimised"}
)

# Résultats
print(f"\nRÉSULTATS XGBOOST")
print(f"AUC CV  : {results['cv_auc_roc_mean']:.4f}")
print(f"AUC Val : {results['val_auc_roc']:.4f}")

NETTOYAGE DES NOMS DE COLONNES POUR XGBOOST
Noms de colonnes nettoyés pour XGBoost
Exemple avant : BUREAU_CREDIT_ACTIVE_<LAMBDA_0>
Exemple après : BUREAU_CREDIT_ACTIVE__LAMBDA_0_


MODELE : XGBoost_Balanced
Run ID : 17f75c14e5874586b99e6ffa3347b000

CROSS-VALIDATION STRATIFIEE (5 folds)
────────────────────────────────────────────────────────────────────────────────
Resultats Cross-Validation (5 folds) :
   - AUC-ROC   : 0.7774 (+/-0.0015)
   - Accuracy  : 0.7664 (+/-0.0022)
   - Precision : 0.1989 (+/-0.0019)
   - Recall    : 0.6256 (+/-0.0019)
   - F1-Score  : 0.3019 (+/-0.0023)

ENTRAINEMENT FINAL
────────────────────────────────────────────────────────────────────────────────
Entrainement sur 246,008 echantillons...
Termine en 75.93s

EVALUATION SUR VALIDATION SET
────────────────────────────────────────────────────────────────────────────────

Metriques Validation (Classe 1: Defaut) :
   - AUC-ROC   : 0.7817
   - Accuracy  : 0.7614
   - Precision : 0.1989
   - Recall    : 0.6457
 

2025/10/09 19:13:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/09 19:13:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


   Scores CV sauvegardes

SAUVEGARDE DU MODELE
────────────────────────────────────────────────────────────────────────────────
   Modele enregistre dans MLflow

RESULTATS FINAUX - XGBoost_Balanced
Temps d'entrainement : 75.93s
Temps optimisation seuil : 0.84s

CROSS-VALIDATION (5 folds stratifies) :
   - AUC-ROC   : 0.7774 (+/-0.0015)
   - Accuracy  : 0.7664 (+/-0.0022)
   - Precision : 0.1989 (+/-0.0019)
   - Recall    : 0.6256 (+/-0.0019)
   - F1-Score  : 0.3019 (+/-0.0023)

VALIDATION SET (Seuil 0.5) :
   - AUC-ROC   : 0.7817
   - Accuracy  : 0.7614
   - Precision : 0.1989
   - Recall    : 0.6457
   - F1-Score  : 0.3041
   - TN/FP/FN/TP : 43,622 / 12,916 / 1,759 / 3,206
   - Cout metier : 30,506

COUT METIER OPTIMISE :
   - Seuil optimal          : 0.48
   - Cout au seuil optimal  : 30,249 (vs 30,506 au seuil 0.5)
   - Economie realisee      : 257 (-0.8%)
   - Recall au seuil optimal: 0.6749
   - Defauts detectes       : 3,351 (vs 3,206)
   - Defauts manques        : 1,614 (vs 1,75

<h3 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">COMPARAISON LIGHTGBM VS XGBOOST</h3>

In [50]:
# COMPARAISON LIGHTGBM VS XGBOOST (OPTIMISATIONS)


print("="*80)
print("COMPARAISON FINALE DES OPTIMISATIONS")
print("="*80)
print()

# Créer tableau comparatif
comparison_optim = {
    'Modèle': ['LightGBM', 'XGBoost'],
    'Coût Baseline': [30876, 30742],
    'Coût Optimisé': [study_lgbm.best_value, study_xgb.best_value],
    'Amélioration (absolu)': [
        30876 - study_lgbm.best_value,
        30742 - study_xgb.best_value
    ],
    'Amélioration (%)': [
        ((30876 - study_lgbm.best_value) / 30876) * 100,
        ((30742 - study_xgb.best_value) / 30742) * 100
    ],
    'Temps optimisation (min)': [
        duration_lgbm / 60,
        duration_xgb / 60
    ],
    'Nombre de trials': [
        len(study_lgbm.trials),
        len(study_xgb.trials)
    ]
}

df_optim = pd.DataFrame(comparison_optim)

print("RÉSULTATS DES OPTIMISATIONS:")
print("-"*80)
print(df_optim.to_string(index=False))
print()

# Identifier le meilleur
best_idx = df_optim['Coût Optimisé'].idxmin()
best_model_name = df_optim.loc[best_idx, 'Modèle']
best_cost = df_optim.loc[best_idx, 'Coût Optimisé']

print("="*80)
print(f"CHAMPION : {best_model_name}")
print("="*80)
print(f"Coût métier optimal : {best_cost:,.0f}")
print()

# Graphique comparatif
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Graphique 1 : Barres de coût
models = ['LightGBM', 'XGBoost']
baseline_costs = [30876, 30742]
optimized_costs = [study_lgbm.best_value, study_xgb.best_value]

x = np.arange(len(models))
width = 0.35

bars1 = axes[0].bar(x - width/2, baseline_costs, width, label='Baseline', color='#3498db', alpha=0.7)
bars2 = axes[0].bar(x + width/2, optimized_costs, width, label='Optimisé Optuna', color='#2ecc71', alpha=0.7)

axes[0].set_xlabel('Modèle', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Coût métier optimal', fontsize=12, fontweight='bold')
axes[0].set_title('Comparaison Baseline vs Optimisé', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Ajouter valeurs sur barres
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:,.0f}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')

# Graphique 2 : Amélioration en %
improvements = [
    ((30876 - study_lgbm.best_value) / 30876) * 100,
    ((30742 - study_xgb.best_value) / 30742) * 100
]

colors_improvement = ['#2ecc71' if imp > 0 else '#e74c3c' for imp in improvements]
bars = axes[1].bar(models, improvements, color=colors_improvement, alpha=0.7, edgecolor='black')

axes[1].set_xlabel('Modèle', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Amélioration (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Amélioration par Optimisation Optuna', fontsize=14, fontweight='bold')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.8)
axes[1].grid(axis='y', alpha=0.3)

# Ajouter valeurs sur barres
for bar, improvement in zip(bars, improvements):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{improvement:.2f}%',
                ha='center', va='bottom' if improvement > 0 else 'top',
                fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('graphiques/comparison_optuna_results.png', dpi=100, bbox_inches='tight')
plt.close()

print("Graphique comparatif sauvegardé : graphiques/comparison_optuna_results.png")
print()

# Sauvegarder résultats
df_optim.to_csv('graphiques/optuna_optimization_results.csv', index=False)
print("Résultats sauvegardés : graphiques/optuna_optimization_results.csv")
print()

print("="*80)
print("OPTIMISATION OPTUNA COMPLÈTE")
print("="*80)
print()
print(f"Durée totale : {(duration_lgbm + duration_xgb)/60:.1f} minutes")
print(f"Fichiers générés : 12 visualisations + 3 fichiers de résultats")
print()

COMPARAISON FINALE DES OPTIMISATIONS

RÉSULTATS DES OPTIMISATIONS:
--------------------------------------------------------------------------------
  Modèle  Coût Baseline  Coût Optimisé  Amélioration (absolu)  Amélioration (%)  Temps optimisation (min)  Nombre de trials
LightGBM          30876        30118.0                  758.0          2.454981                 44.562380                50
 XGBoost          30742        30279.0                  463.0          1.506083                 61.808192                50

CHAMPION : LightGBM
Coût métier optimal : 30,118

Graphique comparatif sauvegardé : graphiques/comparison_optuna_results.png

Résultats sauvegardés : graphiques/optuna_optimization_results.csv

OPTIMISATION OPTUNA COMPLÈTE

Durée totale : 106.4 minutes
Fichiers générés : 12 visualisations + 3 fichiers de résultats



<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Synthèse des résultats</h2>

## **MÉTRIQUES MÉTIER**

| Modèle | Type | Coût Métier<br>Optimal | Seuil<br>Optimal | Défauts<br>Détectés (TP) | Défauts<br>Manqués (FN) | Bons Clients<br>Refusés (FP) | Bons Clients<br>Acceptés (TN) | Taux de<br>Refus Global |
|--------|------|------------------------|------------------|--------------------------|-------------------------|------------------------------|-------------------------------|-------------------------|
| **LightGBM** | **Optimisé** | **30,118** | **0.50** | **3,320** | **1,645** | **13,668** | **42,870** | **24.18%** |
| XGBoost | Optimisé | 30,249 | 0.48 | 3,351 | 1,614 | 14,109 | 42,429 | 24.96% |
| XGBoost | Baseline | 30,742 | 0.50 | 3,246 | 1,719 | 13,552 | 42,986 | 23.98% |
| LightGBM | Baseline | 30,876 | 0.52 | 3,306 | 1,659 | 14,286 | 40,930 | 25.27% |
| LogisticReg | Baseline | 31,408 | 0.53 | 3,261 | 1,704 | 14,368 | 40,122 | 25.41% |
| RandomForest | Baseline | 32,671 | 0.48 | 3,215 | 1,750 | 15,171 | 41,886 | 26.83% |

**Formule du coût :** `Coût Total = (FN × 10) + (FP × 1)`

---

## **MÉTRIQUES TECHNIQUES**

| Modèle | Type | AUC-ROC<br>CV | AUC-ROC<br>Val | Accuracy<br>Val | Precision<br> | Recall<br> | F1-Score<br> | CV Std<br>(AUC) |
|--------|------|---------------|----------------|-----------------|-------------------------|----------------------|------------------------|-----------------|
| **LightGBM** | **Optimisé** | **0.7804** | **0.7828** | 75.10% | 19.54% | 66.87% | 30.25% | **±0.0017** |
| XGBoost | Optimisé | 0.7774 | 0.7817 | **76.14%** | **19.89%** | **67.49%** | **30.41%** | ±0.0015 |
| XGBoost | Baseline | 0.7735 | 0.7772 | 75.17% | 19.32% | 65.38% | 29.83% | ±0.0015 |
| LightGBM | Baseline | 0.7764 | 0.7795 | 72.12% | 18.00% | 69.02% | 28.56% | ±0.0027 |
| LogisticReg | Baseline | 0.7696 | 0.7739 | 70.87% | 17.42% | 69.75% | 27.88% | ±0.0032 |
| RandomForest | Baseline | 0.7500 | 0.7535 | 75.65% | 18.69% | 60.22% | 28.53% | ±0.0028 |

---

## HYPERPARAMÈTRES OPTIMAUX**

### **LightGBM Optimisé**

| Hyperparamètre | Baseline | **Optimisé** | Variation | Impact |
|----------------|----------|--------------|-----------|--------|
| `n_estimators` | 150 | **300** | +100% | Plus d'arbres → meilleure convergence |
| `max_depth` | 7 | **7** | 0% | Profondeur optimale conservée |
| `learning_rate` | 0.050 | **0.0583** | +16.6% | Learning rate légèrement augmenté |
| `num_leaves` | 31 | **40** | +29.0% | Complexité accrue par arbre |
| `min_child_samples` | 20 | **30** | +50.0% | Plus de contraintes → moins d'overfitting |
| `subsample` | 1.0 | **0.7** | -30.0% | Sous-échantillonnage → généralisation |
| `colsample_bytree` | 1.0 | **0.7** | -30.0% | Sélection features → robustesse |
| `reg_alpha` (L1) | 0.0 | **0.9** | +∞ | Régularisation forte ajoutée |
| `reg_lambda` (L2) | 0.0 | **0.8** | +∞ | Régularisation forte ajoutée |
| `class_weight` | balanced | balanced | - | Gestion déséquilibre maintenue |

---

### **XGBoost Optimisé**

| Hyperparamètre | Baseline | **Optimisé** | Variation | Impact |
|----------------|----------|--------------|-----------|--------|
| `n_estimators` | 150 | **300** | +100% | Plus d'arbres → meilleure convergence |
| `max_depth` | 7 | **6** | -14.3% | Moins de profondeur → moins d'overfitting |
| `learning_rate` | 0.050 | **0.0680** | +36.0% | Learning rate augmenté |
| `min_child_weight` | 1 | **7** | +600% | Contraintes fortes → stabilité |
| `gamma` | 0.0 | **0.5** | +∞ | Régularisation par coût de split |
| `subsample` | 1.0 | **1.0** | 0% | Pas de sous-échantillonnage lignes |
| `colsample_bytree` | 1.0 | **0.8** | -20.0% | Sélection features par arbre |
| `colsample_bylevel` | 1.0 | **0.6** | -40.0% | Sélection features par niveau |
| `reg_alpha` (L1) | 0.0 | **0.8** | +∞ | Régularisation L1 ajoutée |
| `reg_lambda` (L2) | 0.0 | **0.1** | +∞ | Légère régularisation L2 |
| `scale_pos_weight` | 11.4 | **11.4** | 0% | Ratio classes maintenu |


<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">LightGBM Feature Importance</h2>

In [24]:
 # SHAP - Feature Importance LightGBM - Log MLflow
with mlflow.start_run(run_name="Feature_Importance_LightGBM"):
    
        # ANALYSE FEATURE IMPORTANCE - LIGHTGBM
    
    print("="*80)
    print("ANALYSE DES FEATURES IMPORTANCE")
    print("="*80)
    print()
    
    # ---------- Chargement du modèle ----------
    
    print("Chargement du modèle LightGBM")
    model_champion = LGBMClassifier(
        n_estimators=300,
        max_depth=7,
        learning_rate=0.058264831213179456,
        num_leaves=40,
        min_child_samples=30,
        subsample=0.7,
        colsample_bytree=0.7,
        reg_alpha=0.9,
        reg_lambda=0.8,
        class_weight='balanced',
        random_state=42,
        force_row_wise=True
    )
    
    # Entraînement sur le dataset complet
    model_champion.fit(X_train_lgbm, y_train)
    print("Modèle entraîné")
    print()
    
    # ---------- FEATURE IMPORTANCE NATIVE (LIGHTGBM) ----------
    
    print("1. FEATURE IMPORTANCE NATIVE (Gain)")
    print("-"*80)
    
    # Récupérer les importances
    feature_importances = pd.DataFrame({
        'feature': X_train_lgbm.columns,
        'importance': model_champion.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Top 20
    print("\nTOP 20 FEATURES LES PLUS IMPORTANTES (Gain):")
    print(feature_importances.head(20).to_string(index=False))
    print()
    
    # Graphique
    fig, ax = plt.subplots(figsize=(12, 8))
    top_20 = feature_importances.head(20)
    ax.barh(range(len(top_20)), top_20['importance'], color='#3498db', alpha=0.7)
    ax.set_yticks(range(len(top_20)))
    ax.set_yticklabels(top_20['feature'])
    ax.set_xlabel('Importance (Gain)', fontsize=12, fontweight='bold')
    ax.set_title('Top 20 Features - LightGBM (Feature Importance Native)', 
                fontsize=14, fontweight='bold')
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig('graphiques/feature_importance_native.png', dpi=100, bbox_inches='tight')
    plt.close()
    print("Graphique sauvegardé : graphiques/feature_importance_native.png")
    print()
    
    # ---------- SHAP VALUES (EXPLAINABILITY) ----------
    
    print("SHAP VALUES (Explainability)")
    print("-"*80)
    print("Calcul des SHAP values (peut prendre 2-3 minutes)...")
    print()
    
    # Créer explainer SHAP (Tree Explainer pour LightGBM)
    explainer = shap.TreeExplainer(model_champion)
    
    # Calculer SHAP values sur un échantillon (10000 samples)
    # Pour l'analyse complète, utiliser tout X_val_lgbm
    sample_size = 10000
    X_sample = X_val_lgbm.sample(n=sample_size, random_state=42)
    shap_values = explainer.shap_values(X_sample)
    
    print(f"SHAP values calculées pour {sample_size} échantillons")
    print()
    
    # SHAP retourne [shap_values_class_0, shap_values_class_1]
    # On utilise shap_values_class_1 (défaut)
    shap_values_class_1 = shap_values[1] if isinstance(shap_values, list) else shap_values
    
    # ---------- SHAP SUMMARY PLOT ----------
    
    print("3. GÉNÉRATION DES VISUALISATIONS SHAP")
    print("-"*80)
    
    # 3.1 Summary Plot (Beeswarm)
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values_class_1, X_sample, 
                     plot_type="dot", show=False, max_display=20)
    plt.title('SHAP Summary Plot - Top 20 Features', 
             fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('graphiques/shap_summary_plot.png', dpi=100, bbox_inches='tight')
    plt.close()
    print("SHAP Summary Plot sauvegardé : graphiques/shap_summary_plot.png")
    
    # 3.2 Feature Importance (SHAP)
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values_class_1, X_sample, 
                     plot_type="bar", show=False, max_display=20)
    plt.title('SHAP Feature Importance - Top 20 Features', 
             fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('graphiques/shap_feature_importance.png', dpi=100, bbox_inches='tight')
    plt.close()
    print("SHAP Feature Importance sauvegardé : graphiques/shap_feature_importance.png")
    
    # ---------- TOP FEATURES ANALYSIS ----------
    
    print()
    print("4. ANALYSE DES TOP FEATURES")
    print("-"*80)
    
    # Calculer l'importance SHAP moyenne (valeur absolue)
    shap_importance = pd.DataFrame({
        'feature': X_sample.columns,
        'shap_importance': np.abs(shap_values_class_1).mean(axis=0)
    }).sort_values('shap_importance', ascending=False)
    
    print("\nTOP 20 FEATURES (SHAP Mean Absolute Value):")
    print(shap_importance.head(20).to_string(index=False))
    print()
    
    # Sauvegarder
    shap_importance.to_csv('graphiques/shap_feature_importance.csv', index=False)
    print("Données sauvegardées : graphiques/shap_feature_importance.csv")
    print()
    
    # ---------- DEPENDENCE PLOTS (Top 3 features) ----------
    
    print("DEPENDENCE PLOTS (Top 3 Features)")
    print("-"*80)
    
    top_3_features = shap_importance.head(3)['feature'].tolist()
    
    for idx, feature in enumerate(top_3_features, 1):
        plt.figure(figsize=(10, 6))
        shap.dependence_plot(
            feature, 
            shap_values_class_1, 
            X_sample, 
            show=False,
            interaction_index=None
        )
        plt.title(f'SHAP Dependence Plot - {feature}', 
                 fontsize=14, fontweight='bold')
        plt.tight_layout()
        filename = f'graphiques/shap_dependence_{idx}_{feature[:30]}.png'
        plt.savefig(filename, dpi=100, bbox_inches='tight')
        plt.close()
        print(f"Dependence plot {idx} sauvegardé : {filename}")
    
    print()
    
    # ---------- WATERFALL PLOT ----------
    
    print("6. WATERFALL PLOT (Explication d'une prédiction)")
    print("-"*80)
    
    # Prendre un exemple de client avec défaut (classe 1)
    idx_defaut = y_val.iloc[:sample_size].reset_index(drop=True)
    idx_defaut = idx_defaut[idx_defaut == 1].index[0]
    
    print(f"Exemple: Client avec défaut (index {idx_defaut})")
    print()
    
    # Waterfall plot
    plt.figure(figsize=(12, 8))
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values_class_1[idx_defaut],
            base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
            data=X_sample.iloc[idx_defaut],
            feature_names=X_sample.columns
        ),
        max_display=15,
        show=False
    )
    plt.title('SHAP Waterfall Plot - Explication prédiction client défaut', 
             fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('graphiques/shap_waterfall_defaut.png', dpi=100, bbox_inches='tight')
    plt.close()
    print("Waterfall plot sauvegardé : graphiques/shap_waterfall_defaut.png")
    print()
    
    # ---------- RÉCAPITULATIF ----------
    
    print("="*80)
    print("ANALYSE FEATURE IMPORTANCE TERMINÉE")
    print("="*80)
    print()
    print("FICHIERS GÉNÉRÉS:")
    print("  graphiques/feature_importance_native.png (LightGBM native)")
    print("  graphiques/shap_summary_plot.png (Beeswarm)")
    print("  graphiques/shap_feature_importance.png (Bar chart)")
    print("  graphiques/shap_feature_importance.csv (Données)")
    print("  graphiques/shap_dependence_1_*.png (Top 1)")
    print("  graphiques/shap_dependence_2_*.png (Top 2)")
    print("  graphiques/shap_dependence_3_*.png (Top 3)")
    print("  graphiques/shap_waterfall_defaut.png (Exemple)")
    print()
    print("INTERPRÉTATION:")
    print("  • SHAP Summary Plot : Montre l'impact de chaque feature")
    print("    - Rouge = Valeur élevée de la feature")
    print("    - Bleu = Valeur faible de la feature")
    print("    - Axe X = Impact sur la prédiction (+ = augmente risque défaut)")
    print()
    print("  • Dependence Plot : Relation entre feature et prédiction")
    print("    - Axe X = Valeur de la feature")
    print("    - Axe Y = SHAP value (impact)")
    print()
    print("  • Waterfall Plot : Décomposition d'une prédiction individuelle")
    print("    - Base value = Prédiction moyenne")
    print("    - Chaque barre = Contribution d'une feature")
    print("    - Final value = Prédiction pour ce client")
    print()
    
    # ---------- LOGGER DANS MLFLOW ----------
    
    print()
    print("="*80)
    print("LOGGING DANS MLFLOW")
    print("="*80)
    print()
    
    # Logger les artifacts individuels
    mlflow.log_artifact('graphiques/feature_importance_native.png')
    mlflow.log_artifact('graphiques/shap_summary_plot.png')
    mlflow.log_artifact('graphiques/shap_feature_importance.png')
    mlflow.log_artifact('graphiques/shap_feature_importance.csv')
    mlflow.log_artifact('graphiques/shap_waterfall_defaut.png')
    
    # Logger les dependence plots (glob pour trouver les fichiers)
    import glob
    dependence_files = glob.glob('graphiques/shap_dependence_*.png')
    for file in dependence_files:
        mlflow.log_artifact(file)
        print(f"Loggé : {file}")
    
    print()
    
    # Logger les paramètres
    mlflow.log_param("model_name", "LightGBM_Finale_1.0")
    mlflow.log_param("shap_sample_size", 10000)
    mlflow.log_param("top_features", 20)
    mlflow.log_param("n_estimators", 300)
    mlflow.log_param("max_depth", 7)
    mlflow.log_param("learning_rate", 0.058264831213179456)
    
    # Logger les métriques importantes du top 3
    for i, row in shap_importance.head(3).iterrows():
        mlflow.log_metric(f"shap_importance_{row['feature'][:50]}", row['shap_importance'])
    
    # Tags
    mlflow.set_tags({
        "type": "Feature_importance",
        "method": "SHAP",
        "model_family": "LightGBM"
    })
    
    print(" Tous les artifacts et métadonnées loggés dans MLflow")
    print()

ANALYSE DES FEATURES IMPORTANCE

Chargement du modèle LightGBM
[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Total Bins 105842
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 876
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Modèle entraîné

1. FEATURE IMPORTANCE NATIVE (Gain)
--------------------------------------------------------------------------------

TOP 20 FEATURES LES PLUS IMPORTANTES (Gain):
                       feature  importance
  

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

# ANALYSE FEATURES IMPORTANCE - LIGHTGBM

---

## TOP 10 FEATURES LES PLUS IMPORTANTES (SHAP)

| Rang | Feature | SHAP Importance | Type | Interprétation |
|------|---------|-----------------|------|----------------|
| **1** | **EXT_SOURCE_2 × EXT_SOURCE_3** | **0.2915** | Interaction | Combinaison des scores externes 2 et 3 (sources de crédit) |
| **2** | **EXT_SOURCE_1 × EXT_SOURCE_3** | **0.1602** | Interaction | Combinaison des scores externes 1 et 3 |
| **3** | **AMT_ANNUITY** | **0.1049** | Montant | Montant de l'annuité du crédit |
| **4** | **EXT_SOURCE_1 × EXT_SOURCE_2** | **0.1030** | Interaction | Combinaison des scores externes 1 et 2 |
| **5** | **EXT_SOURCE_1 × DAYS_BIRTH** | **0.0904** | Interaction | Score externe 1 combiné avec l'âge |
| **6** | **DAYS_EMPLOYED** | **0.0894** | Temporel | Nombre de jours d'emploi |
| **7** | **AMT_GOODS_PRICE** | **0.0881** | Montant | Prix des biens achetés |
| **8** | **AMT_CREDIT** | **0.0872** | Montant | Montant du crédit |
| **9** | **NAME_EDUCATION_Higher** | **0.0710** | Catégoriel | Éducation supérieure (binaire) |
| **10** | **INST_PAYMENT_DIFF_MEAN** | **0.0665** | Historique | Différence moyenne paiements prévus/réels |

---

## INSIGHTS CLÉS PAR CATÉGORIE

### 1. FEATURES EXTERNES (EXT_SOURCE) -  Dominantes

**Top 3 = Interactions polynomiales (Feature Engineering)**

| Feature | Impact | Observation SHAP |
|---------|--------|------------------|
| EXT_SOURCE_2 × EXT_SOURCE_3 | **+++++** | **Valeurs élevées → FORTE réduction risque défaut** |
| EXT_SOURCE_1 × EXT_SOURCE_3 | **++++** | Relation inverse forte avec défaut |
| EXT_SOURCE_1 × EXT_SOURCE_2 | **++++** | Effet protecteur important |

**Interprétation :**
- Les **scores externes** sont des évaluations de solvabilité par organismes tiers
- Plus le score est **élevé** → Plus le client est **fiable**
- Les **interactions** capturent des effets non-linéaires mieux que les variables seules

---

### 2. MONTANTS FINANCIERS - Critiques

| Feature | SHAP | Relation avec Défaut |
|---------|------|----------------------|
| **AMT_ANNUITY** | 0.1049 | **Annuités élevées** → Risque accru (difficulté remboursement) |
| **AMT_GOODS_PRICE** | 0.0881 | Prix élevés → Risque accru |
| **AMT_CREDIT** | 0.0872 | Crédits importants → Risque accru |

**Pattern identifié :** Plus les montants sont élevés, plus le risque augmente (capacité de remboursement limitée).

---

### 3. PROFIL CLIENT - Déterminant

| Feature | SHAP | Impact |
|---------|------|--------|
| **NAME_EDUCATION_Higher** | 0.0710 | Éducation supérieure **réduit** le risque |
| **NAME_FAMILY_STATUS_Married** | 0.0639 | Marié **réduit** le risque (stabilité) |
| **CODE_GENDER_M** | 0.0597 | Homme légèrement plus risqué |
| **CODE_GENDER_F** | 0.0584 | Femme légèrement moins risquée |

**Insight métier :** Profil socio-démographique stable = moins de défauts.

---

### 4. HISTORIQUE FINANCIER - Important

| Feature | SHAP | Source | Interprétation |
|---------|------|--------|----------------|
| **DAYS_EMPLOYED** | 0.0894 | Application | Ancienneté emploi (stabilité professionnelle) |
| **INST_PAYMENT_DIFF_MEAN** | 0.0665 | Installments | Écart moyen paiements → Régularité |
| **BUREAU_AMT_CREDIT_SUM_DEBT_STD** | 0.0490 | Bureau | Volatilité dettes passées |
| **BUREAU_DAYS_ENDDATE_FACT_MAX** | 0.0445 | Bureau | Date fin réelle derniers crédits |

**Pattern :** Historique de paiement régulier = fiabilité accrue.

---

### 5. FEATURES AGRÉGÉES (Bureau, Previous) -  Complémentaires

**Top Features Agrégées :**
- **BUREAU** : Historique crédit externe (6 features dans TOP 20 SHAP)
- **PREV** : Demandes précédentes Home Credit (4 features dans TOP 20)
- **INST** : Historique paiements installments (2 features dans TOP 20)

**Validation :** Le feature engineering (agrégations) apporte de la valeur prédictive.

---

## INTERPRÉTATION DES GRAPHIQUES SHAP

### Dependence Plot : EXT_SOURCE_2 × EXT_SOURCE_3

**Observation :**
```
Valeur = 0.0-0.1 → SHAP = +1.0 (AUGMENTE défaut)
Valeur = 0.4-0.7 → SHAP = -0.5 (RÉDUIT défaut)
```

**Conclusion :** Plus le produit des scores externes est élevé, plus le client est fiable (relation inverse forte).

---

### Dependence Plot : AMT_ANNUITY

**Observation :**
```
Annuité < 30,000  → SHAP négatif (bon signe)
Annuité > 50,000  → SHAP positif (risque accru)
```

**Conclusion :** Les annuités trop élevées augmentent le risque de défaut (charge financière).

---

### Summary Plot (Beeswarm)

**Légende des couleurs :**
- **Rouge** = Valeur élevée de la feature
- **Bleu** = Valeur faible de la feature

**Patterns identifiés :**
1. **EXT_SOURCE élevés (rouge)** → SHAP négatif → **Protège contre défaut**
2. **AMT_ANNUITY élevé (rouge)** → SHAP positif → **Augmente défaut**
3. **Higher Education (rouge)** → SHAP négatif → **Protège contre défaut**

---

## RECOMMANDATIONS MÉTIER

### 1. Priorité absolue : Scores externes
- **Obtenir EXT_SOURCE_1/2/3** pour TOUS les clients
- Si manquants → Imputer ou obtenir via partenariats
- Les interactions (×) sont **2× plus importantes** que les variables seules

### Seuils d'alerte montants
- **AMT_ANNUITY > 50,000** → Risque élevé (vérification manuelle)
- **AMT_CREDIT > 1,500,000** → Risque accru
- **AMT_GOODS_PRICE > 1,800,000** → Attention renforcée

### Profils à favoriser
- Éducation supérieure
- Statut marié
- Ancienneté emploi > 5 ans (1,825 jours)
- Historique paiements réguliers

### 4. Red flags historique
- Volatilité dettes passées élevée (BUREAU_STD)
- Écarts importants paiements prévus/réels (INST_PAYMENT_DIFF)
- Taux refus élevé demandes précédentes (PREV)

---

## FEATURE IMPORTANCE : NATIVE vs SHAP

| Méthode | Top 1 Feature | Différence Clé |
|---------|---------------|----------------|
| **LightGBM Native** | AMT_CREDIT (192) | Mesure "splits" → Fréquence utilisation |
| **SHAP** | EXT_SOURCE_2×3 (0.29) | Mesure "impact" → Importance prédictive réelle |

**SHAP > Native** : Capture mieux l'impact réel sur les prédictions.

---

## WATERFALL PLOT - EXEMPLE CLIENT DÉFAUT

**Client analysé : f(x) = -0.249** (probabilité défaut inférieure à la moyenne)

**Top contributions :**
1. **EXT_SOURCE_2×3 = 0.093** → **+0.53** (augmente défaut) 
2. **EXT_SOURCE_1×3 = 0.079** → **+0.29** (augmente défaut)
3. **AMT_GOODS_PRICE = 1.8M** → **-0.21** (réduit défaut)
4. **DAYS_EMPLOYED = -4,685** → **-0.16** (réduit défaut)

**Interprétation :** Client avec scores externes FAIBLES (risque) mais emploi stable et montants raisonnables (compense).

---

## SYNTHÈSE CHIFFRÉE

| Catégorie | Nombre Features TOP 20 | % Importance Cumulée |
|-----------|------------------------|----------------------|
| **EXT_SOURCE (interactions)** | 5 | **45%**  |
| **Montants (AMT_*)** | 3 | **25%** |
| **Profil client** | 4 | **15%** |
| **Historique (BUREAU/INST)** | 5 | **10%** |
| **Autres** | 3 | **5%** |

---

## CONCLUSION

### 3 Pilliers de la Prédiction :

1. **Scores externes** (45% importance) → Obtenir pour tous clients
2. **Montants financiers** (25% importance) → Surveillance seuils
3. **Profil socio-démographique** (15% importance) → Favoriser stabilité

**Le feature engineering (interactions polynomiales) a été DÉCISIF :** Les 3 features les plus importantes sont des interactions créées manuellement !

---

## FICHIERS GÉNÉRÉS

**Artifacts disponibles dans MLflow :**
- `feature_importance_native.png` - Importance native LightGBM
- `shap_summary_plot.png` - Beeswarm plot (vue d'ensemble)
- `shap_feature_importance.png` - Bar chart SHAP
- `shap_feature_importance.csv` - Données complètes (912 features)
- `shap_dependence_1_EXT_SOURCE_2_EXT_SOURCE_3.png` - Dependence plot #1
- `shap_dependence_2_EXT_SOURCE_1_EXT_SOURCE_3.png` - Dependence plot #2
- `shap_dependence_3_AMT_ANNUITY.png` - Dependence plot #3
- `shap_waterfall_defaut.png` - Explication client exemple

---

**Analyse terminée - LightGBM Champion optimisé**  
**Coût métier : 30,118 | AUC : 0.7828 | Features analysées : 912**

<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">MODEL REGISTRY - LIGHTGBM</h2>

In [29]:
# MODEL REGISTRY 
# Configuration
mlflow.set_experiment("home_credit_scoring")

# Nom du modèle dans le registry
model_name = "lightgbm_optimised_1"

# Créer un run et enregistrer directement dans le registry
with mlflow.start_run(run_name="LightGBM_optimisé_métier") as run:
    
    # Logger les hyperparamètres
    mlflow.log_params({
        "n_estimators": 300,
        "max_depth": 7,
        "learning_rate": 0.058264831213179456,
        "num_leaves": 40,
        "min_child_samples": 30,
        "subsample": 0.7,
        "colsample_bytree": 0.7,
        "reg_alpha": 0.9,
        "reg_lambda": 0.8,
        "class_weight": "balanced",
        "random_state": 42
    })
    
    # Logger ET enregistrer dans le registry
    mlflow.sklearn.log_model(
        sk_model=model_champion,
        artifact_path="model",
        registered_model_name=model_name
    )

    # Tags
    mlflow.set_tags({
        "mlflow.user": "Mounir Meknaci",
        "model_registry": "LightGBM 1.0",
        "business_threshold": "Optimized"
    })
        
    print(f"Modèle loggé et enregistré : {model_name}")

# Mettre en Production
client = MlflowClient()
latest_version = client.get_latest_versions(model_name, stages=["None"])[0]

client.transition_model_version_stage(
    name=model_name,
    version=latest_version.version,
    stage="Production"
)

print(f"Version {latest_version.version} en Production")
print(f"Accès : models:/{model_name}/Production")

# ---------- Description ----------

description = """
# LightGBM Optimisé - Scoring Crédit

## Vue d'ensemble
Modèle de classification binaire optimisé pour prédire le risque de défaut de paiement dans le cadre de demandes de crédit à la consommation. Ce modèle a été spécifiquement conçu pour minimiser le coût métier en tenant compte du déséquilibre entre Faux Négatifs (10x) et Faux Positifs (1x).

## Caractéristiques Techniques
- **Algorithme** : LightGBM (Gradient Boosting)
- **Méthode d'optimisation** : Optuna (50 trials)
- **Critère optimisé** : Coût métier total (FN×10 + FP×1)
- **Validation** : Hold-out stratifié (80/20) + Cross-validation 5 folds
- **Features** : 912 (agrégations Bureau/Previous + interactions polynomiales)
- **Class handling** : Balanced weights

## Performances
- **AUC-ROC** : ~0.78
- **Coût métier optimal** : ~30,000 (seuil optimisé)
- **Seuil de décision** : Optimisé dynamiquement (≠ 0.5)

```

## Hyperparamètres Clés
- **n_estimators** : 300
- **max_depth** : 7
- **learning_rate** : 0.0583
- **num_leaves** : 40
- **Régularisation** : L1=0.9, L2=0.8

## Top 3 Features (SHAP)
1. **EXT_SOURCE_2 × EXT_SOURCE_3** (0.29) - Scores solvabilité
2. **EXT_SOURCE_1 × EXT_SOURCE_3** (0.16) - Interactions externes
3. **AMT_ANNUITY** (0.10) - Montant annuité

## Informations
- **Version** : 1.0
- **Date** : Octobre 2025
- **Projet** : Home Credit Default Risk

Mounir Meknaci - Data Scientist
"""

# Ajouter la description au modèle
client.update_registered_model(
    name=model_name,
    description=description
)

2025/10/20 14:56:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/20 14:56:24 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/20 14:56:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle loggé et enregistré : lightgbm_optimised_1
Version 1 en Production
Accès : models:/lightgbm_optimised_1/Production


Successfully registered model 'lightgbm_optimised_1'.
Created version '1' of model 'lightgbm_optimised_1'.


<RegisteredModel: aliases={}, creation_timestamp=1760964984504, deployment_job_id=None, deployment_job_state=None, description=('\n'
 '# LightGBM Optimisé - Scoring Crédit\n'
 '\n'
 "## Vue d'ensemble\n"
 'Modèle de classification binaire optimisé pour prédire le risque de défaut '
 'de paiement dans le cadre de demandes de crédit à la consommation. Ce modèle '
 'a été spécifiquement conçu pour minimiser le coût métier en tenant compte du '
 'déséquilibre entre Faux Négatifs (10x) et Faux Positifs (1x).\n'
 '\n'
 '## Objectif Métier\n'
 'Identifier automatiquement les clients à risque de défaut afin de :\n'
 '- Réduire les pertes liées aux crédits non remboursés\n'
 '- Minimiser le refus de bons clients (manque à gagner)\n'
 '- Optimiser le seuil de décision selon le coût métier\n'
 '\n'
 '## Caractéristiques Techniques\n'
 '- **Algorithme** : LightGBM (Gradient Boosting)\n'
 "- **Méthode d'optimisation** : Optuna (50 trials)\n"
 '- **Critère optimisé** : Coût métier total (FN×10 + FP×